In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# %cd /content/gdrive/MyDrive/DATASETS/CSE-CIC-IDS-2018_KAGGLE/CSE_CIC_IDS_ALL

In [ ]:
%cd /content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/

/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018


In [ ]:
!ls -lrt

total 3288426
-rw-------  1 root root    104482 Feb  5  2024 'SQL Injection.csv'
-rw-------  1 root root  14111086 Feb  5  2024 'DoS attacks-Slowloris.csv'
-rw-------  1 root root   2093031 Feb  5  2024 'DDOS attack-LOIC-UDP.csv'
-rw-------  1 root root    273431 Feb  5  2024 'Brute Force -XSS.csv'
-rw-------  1 root root    770429 Feb  5  2024 'Brute Force -Web.csv'
-rw-------  1 root root  52375241 Feb  5  2024 'DoS attacks-GoldenEye.csv'
-rw-------  1 root root 155416507 Feb  5  2024 'DoS attacks-SlowHTTPTest.csv'
-rw-------  1 root root 365072331 Feb  5  2024  Bot.csv
-rw-------  1 root root 804021836 Feb  5  2024 'DDOS attack-HOIC.csv'
-rw-------  1 root root 194111176 Feb  5  2024  Infilteration.csv
-rw-------  1 root root 218035465 Feb  5  2024  FTP-BruteForce.csv
-rw-------  1 root root 526095460 Feb  5  2024 'DoS attacks-Hulk.csv'
-rw-------  1 root root 803522632 Feb  5  2024 'DDoS attacks-LOIC-HTTP.csv'
-rw-------  1 root root 231234571 Feb  5  2024  SSH-Bruteforce.csv
drwx-

In [ ]:
# -*- coding: utf-8 -*-
# Bilevel IDS training (inner BCE on normal-logit, outer ODIN BCE) – checklist-complete edition.

import os, gc, glob, math, random
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import os
import matplotlib
matplotlib.use("Agg")  # force non-GUI backend so nothing tries to display

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import (
    f1_score, precision_score, recall_score, accuracy_score,
    average_precision_score, precision_recall_curve, auc, roc_auc_score
)

In [ ]:
import os
from typing import Dict, Tuple
import re

# ---------- robust path resolve (fix common hyphen/underscore mixups) ----------
def resolve_data_dir(path: str) -> str:
    cand = os.path.expanduser(path.rstrip("/ "))
    if os.path.isdir(cand):
        return cand
    # try common variants
    variants = [
        cand.replace("CSE-CIC-IDS_ALL_DATA", "CSE_CIC_IDS_ALL_DATA"),
        cand.replace("CSE_CIC_IDS_ALL_DATA", "CSE-CIC-IDS_ALL_DATA"),
    ]
    for v in variants:
        if os.path.isdir(v):
            return v
    raise FileNotFoundError(f"Directory not found: {path}\nTried: {cand}\n"
                            f"Hint: check underscores vs hyphens in 'CSE_CIC_IDS_ALL_DATA'.")

# ---------- filename cleaning for category/subcategory ----------
def _clean_display_name(name: str) -> str:
    s = name.strip()
    if len(s) >= 2 and s[0] == s[-1] and s[0] in ("'", '"'):
        s = s[1:-1]
    return s

def _parse_cat_subcat_from_display_name(clean_name: str) -> Tuple[str, str]:
    parts = re.split(r"[-–—_]", clean_name, maxsplit=1)
    if len(parts) == 2:
        return parts[0].strip(), parts[1].strip()
    return clean_name.strip(), ""

def discover_files_by_category(data_dir: str) -> Dict[str, str]:
    """
    Finds CSVs case-insensitively, robust to spaces/quotes in names.
    Key is "Category::Subcategory" (or just "Category" if none).
    """
    data_dir = resolve_data_dir(data_dir)
    out: Dict[str, str] = {}
    with os.scandir(data_dir) as it:
        for entry in it:
            if entry.is_file() and entry.name.lower().endswith(".csv"):
                cleaned = _clean_display_name(os.path.splitext(entry.name)[0])
                cat, sub = _parse_cat_subcat_from_display_name(cleaned)
                fam_key = f"{cat}::{sub}" if sub else cat
                out[fam_key] = entry.path  # exact filesystem path
    return out

# ---------- USE IT ----------
DATA_DIR = "/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/"
DATA_DIR = resolve_data_dir(DATA_DIR)  # will raise with a helpful message if wrong

files = discover_files_by_category(DATA_DIR)
if not files:
    raise RuntimeError(f"No CSVs found in {DATA_DIR}")

print("Discovered families:")
for k, v in sorted(files.items()):
    print(f"  {k} -> {repr(v)}")


Discovered families:
  Bot -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/Bot.csv'
  Brute Force::Web -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/Brute Force -Web.csv'
  Brute Force::XSS -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/Brute Force -XSS.csv'
  DDOS attack::HOIC -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/DDOS attack-HOIC.csv'
  DDOS attack::LOIC-UDP -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/DDOS attack-LOIC-UDP.csv'
  DDoS attacks::LOIC-HTTP -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/DDoS attacks-LOIC-HTTP.csv'
  DoS attacks::GoldenEye -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/DoS attacks-GoldenEye.csv'
  DoS attacks::Hulk -> '/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Pr

In [ ]:
# ===== hardness tracking & mixed sampling ====================================
import numpy as np

class HardnessBook:
    """
    Tracks a rolling hardness per family using per-family VAL stats on TRAIN families.
    Hardness ↑ means 'harder to separate (ID vs OOD)'.
    """
    def __init__(self, fams: list[str], ema=0.6):
        self.ema = float(ema)
        self.h = {f: 1.0 for f in fams}   # start neutral
        self._seen = set(fams)

    @staticmethod
    def _score_one(stats: dict) -> float:
        # Be robust to missing keys
        auc  = float(stats.get("auc_roc", stats.get("auc", 0.5)))
        f1   = float(stats.get("f1", 0.5))
        thr  = float(stats.get("thr", 0.0))
        # Smaller thr* (≈0) often means OOD sits very close to ID ⇒ harder.
        # Compose a bounded hardness: 1-AUC + 0.5*(1-F1) + 0.5*exp(-10*thr)
        return (1.0 - auc) + 0.5*(1.0 - f1) + 0.5*np.exp(-10.0*thr)

    def update_from_perfam(self, perfam_stats: dict):
        for fam, st in perfam_stats.items():
            if fam not in self._seen:   # only train fams; ignore held-out
                continue
            s_old = self.h.get(fam, 1.0)
            s_new = self._score_one(st)
            self.h[fam] = self.ema * s_old + (1.0 - self.ema) * s_new

    def snapshot(self) -> dict[str, float]:
        return dict(self.h)


def select_outer_fams_mixed(
    seen_fams: list[str],
    hardness_now: dict[str, float],
    rng: np.random.Generator,
    k: int,
    hard_frac: float = 0.5,
    hard_quantile: float = 0.5
) -> list[str]:
    """
    Pick k train families with a mixture of 'hard' and 'easy' according to hardness_now.
    hard_quantile=0.5 => top 50% considered 'hard pool'.
    """
    if k <= 0 or len(seen_fams) == 0:
        return []

    scores = np.array([hardness_now.get(f, 1.0) for f in seen_fams], dtype=float)
    q = np.quantile(scores, 1.0 - float(hard_quantile))  # threshold into hard/easy
    hard_pool = [f for f, s in zip(seen_fams, scores) if s >= q]
    easy_pool = [f for f, s in zip(seen_fams, scores) if s <  q]

    rng.shuffle(hard_pool); rng.shuffle(easy_pool)

    h = min(int(round(k * hard_frac)), len(hard_pool))
    e = k - h
    take_h = hard_pool[:h]
    take_e = easy_pool[:e] if e > 0 else []
    sel = (take_h + take_e)[:k]
    rng.shuffle(sel)
    return sel


In [ ]:


# ------------------------ Config ------------------------
#DATA_DIR = "/content/gdrive/MyDrive/DATASETS/CSE-CIC-IDS-2018_KAGGLE/CSE_CIC_IDS_ALL/processed"

DATA_DIR = "/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/"

# Episodes
N_EPISODES   = 20
INNER_EPOCHS = 10
OUTER_EPOCHS = 15

INNER_MAX_FAMS = 2
OUTER_MAX_FAMS = 2         # <-- allow 2 OOD families / batch

INNER_POS_PER_FAM          = 20000
INNER_NEG_PER_FAM          = "match"
OUTER_OOD_POS_PER_FAM      = 2000
OUTER_ID_NEG_PER_INNER_FAM = 20000

INNER_BATCH = 256
OUTER_BATCH = 128

INNER_EPISODE_MAX = None
OUTER_EPISODE_MAX = None

# ----- new knobs (set your defaults) -----------------------------------------
OUTER_TASKS_PER_EP      = 2      # how many outer tasks per episode
OUTER_FAMS_PER_TASK     = 3      # num families in each outer task
OUTER_HARD_FRAC         = 0.5    # fraction of "hard" families per task
OUTER_HARD_QUANTILE     = 0.5    # top-q considered 'hard'
OUTER_COMBINE_TASKS     = True   # if True, combine all tasks into one batch
HARDNESS_EMA            = 0.6    # smoothing for hardness tracking
VAL_CAP_PER_CLASS_TRAIN = 6000   # for per-family train-val (to compute hardness)


#HELDOUT_FAMILIES: List[str] = ["Web1", "DDoS1", "Infil2", "DoS2", "Bruteforce"]

#HELDOUT_FAMILIES: List[str] = ['Brute Force::Web', 'Brute Force::XSS', 'DDOS attack::HOIC', "Infilteration"]

import os, re, unicodedata, difflib


def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKC", s or "")
    s = s.replace("\u00A0", " ")
    s = re.sub(r"[‐-‒–—−]+", "-", s)
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s

def resolve_heldout(user_keys, discovered_keys):
    # normalize discovered keys -> original
    norm2orig = {_norm(k): k for k in discovered_keys}
    resolved, missing = [], []
    for k in user_keys:
        nk = _norm(k)
        if nk in norm2orig:
            resolved.append(norm2orig[nk])
        else:
            # suggest close matches
            cand = difflib.get_close_matches(nk, list(norm2orig.keys()), n=3, cutoff=0.6)
            missing.append((k, [norm2orig[c] for c in cand]))
    return resolved, missing


# ---------------- Example wiring ----------------

# after you’ve discovered files (keys) with your discover_files_by_category():
files = discover_files_by_category(DATA_DIR)   # dict: {family_key: path}
discovered = sorted(files.keys())
print("Discovered families:")
for k in discovered:
    print("  ", k)

# your intended held-out list (as you like to type it):
# HELDOUT_FAMILIES = [
#     'Brute Force::Web',
#     'Brute Force::XSS',
#     'DDOS attack::HOIC',
#     'Infilteration',
#     'DoS attacks::GoldenEye',
#     'FTP::BruteForce',
#     'DoS attacks::Slowloris'
# ]


HELDOUT_FAMILIES = [
    'Brute Force::Web',
    'DDOS attack::HOIC',
    'DoS attacks::GoldenEye',
    'FTP::BruteForce',

]



# resolve to the exact discovered keys
heldout_resolved, missing = resolve_heldout(HELDOUT_FAMILIES, discovered)

if missing:
    print("\nCould not resolve some HELDOUT_FAMILIES exactly:")
    for orig, hints in missing:
        if hints:
            print(f"  - '{orig}'  -> did you mean: {hints}")
        else:
            print(f"  - '{orig}'  -> no close match found")
    # you can choose to raise here if you want:
    # raise ValueError("Fix HELDOUT_FAMILIES entries above.")

# use the resolved names going forward
heldout_fams = heldout_resolved
seen_train_fams = [f for f in discovered if f not in heldout_fams]

# hardness tracker (only for TRAIN families)
hardbook = HardnessBook(fams=seen_train_fams, ema=HARDNESS_EMA)


print("\nHeld-out (eval-only):", heldout_fams)
print("Seen train families :", seen_train_fams)


VAL_PER_CLASS_CAP  = 25_000
TEST_PER_CLASS_CAP = 25_000

ACCUM_STEPS     = 1
USE_AMP         = True
CLIP_NORM       = 1.0
WEIGHT_DECAY    = 1e-4
LABEL_SMOOTH    = 0.02

NUM_WORKERS     = 0
PIN_MEMORY      = False
INNER_LR_MAX    = 3e-3

# Optional reconstruction weight
LAMBDA_REC = 0.10

# Model dims
EMBED_DIM  = 128
ENC_HIDDEN = 256
ENC_DEPTH  = 3
DEC_HIDDEN = 128
DEC_DEPTH  = 2
DROPOUT    = 0.10

# ODIN config (learn both T and eps)
TEMP_INIT       = 1.0
EPS_INIT        = 2e-3
TUNE_TEMPERATURE= True
TUNE_EPSILON    = True

# ODIN step control
TRAIN_PGD_STEPS = 5
VAL_PGD_STEPS   = 1      # speed up validation
ODIN_TAU        = 1e-2   # EMA pullback

# Clamp ranges
T_MIN,  T_MAX    = 0.3, 1.05
EPS_MIN, EPS_MAX = 5e-4, 3e-2

# Robust-threshold building
EASY_AP_CUTOFF = 0.98   # families with AP >= this are "easy" and excluded from robust median


# ----- Calibration controls -----
CALIB_USE_MIN_PREC = True   # if True, pick per-family thr at F1* subject to Precision >= CALIB_MIN_PREC
CALIB_MIN_PREC     = 0.80   # minimum precision constraint for calibration
CALIB_THR_FLOOR    = 1e-3   # never allow global thr below this floor


# ----- Outer margin ("hard separation") -----
OUTER_MARGIN_M     = 0.8    # desired gap: logit_norm(ID) - logit_norm(OOD) >= M
OUTER_MARGIN_ALPHA = 0.4   # strength of the margin penalty


# Prioritize tough seen families for outer OOD (held-out families never appear here)
# HARD_OOD_FAMS = ["Infil1", "DoS1", "Web2"]  # order = priority; will auto-filter to allowed
# TARGET_OOD_K   = 3                           # try to keep 2–3 OOD families (capped by OUTER_MAX_FAMS)

# OUTER_MAX_FAMS = 3            # was likely 2
# HARD_OOD_FAMS = ["Infil1", "Infil2" , "DoS1", "Web2"]  # order is a seed; we’ll rotate it
# TARGET_OOD_K  = 3
# _rr_ptr = 0                   # module-level or kept in your runner's state

# ===== Few-shot OUTER knobs (NEW) =====
FS_NUM_OOD_CLASSES     = 4    # how many OOD families per outer step
FS_K_PER_CLASS_OUTER   = 100   # anomalies per fam, and equal number of normals from same fam
FS_ALLOW_SMALLER       = True # if a fam lacks K, take what’s available

# (kept) “hard” families preference for choosing candidates
OUTER_MAX_FAMS = 3
TARGET_OOD_K   = 3
#HARD_OOD_FAMS  = ["Infil1", "Infil2", "DoS1", "Web1", "Web2"]
HARD_OOD_FAMS= []
rr_ptr = 0

Discovered families:
   Bot
   Brute Force::Web
   Brute Force::XSS
   DDOS attack::HOIC
   DDOS attack::LOIC-UDP
   DDoS attacks::LOIC-HTTP
   DoS attacks::GoldenEye
   DoS attacks::Hulk
   DoS attacks::SlowHTTPTest
   DoS attacks::Slowloris
   FTP::BruteForce
   Infilteration
   SQL Injection
   SSH::Bruteforce

Held-out (eval-only): ['Brute Force::Web', 'DDOS attack::HOIC', 'DoS attacks::GoldenEye', 'FTP::BruteForce']
Seen train families : ['Bot', 'Brute Force::XSS', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'Infilteration', 'SQL Injection', 'SSH::Bruteforce']


In [ ]:



# # ===== Few-shot OOD controls =====
# FS_NUM_OOD_CLASSES =

# FS_K_PER_CLASS_OUTER = 500


# ------------------------ Globals ------------------------
SEED = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed: int = SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

def discover_files(data_dir: str) -> Dict[str, str]:
    paths = sorted(glob.glob(os.path.join(data_dir, "*.csv")))
    out = {}
    for p in paths:
        if os.path.isdir(p):  # guard
            continue
        name = os.path.basename(p)
        fam  = name.split("-")[0]
        out[fam] = p
    return out

def _find_label_column(df: pd.DataFrame) -> str:
    for cand in ["Label", "label", "labels", "y", "target", "class", "Class"]:
        if cand in df.columns: return cand
    last = df.columns[-1]
    if set(pd.unique(df[last]).tolist()).issubset({0,1}): return last
    raise RuntimeError("Could not find binary label column (0/1).")




def safe_savefig(fig, out_path: str):
    """Create parent dirs, save fig, and close it. Robust to missing dirs."""
    import os, matplotlib.pyplot as plt
    out_dir = os.path.dirname(out_path)
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)
    # save and close (no bbox_inches="tight" here!)
    fig.savefig(out_path, dpi=150, bbox_inches=None, pad_inches=0.05)
    plt.close(fig)

In [ ]:
import os, re
from typing import Dict, Tuple

def _clean_display_name(name: str) -> str:
    """
    Clean only for building the family key (NOT for path):
      - strip leading/trailing whitespace
      - strip wrapping single/double quotes if present
    """
    s = name.strip()
    if (len(s) >= 2) and ((s[0] == s[-1]) and s[0] in ("'", '"')):
        s = s[1:-1]
    return s

def _parse_cat_subcat_from_display_name(clean_name: str) -> Tuple[str, str]:
    """
    From a cleaned base name (without extension), extract (category, subcategory).
    Split on the FIRST '-', '–', '—', or '_' only.
    """
    parts = re.split(r"[-–—_]", clean_name, maxsplit=1)
    if len(parts) == 2:
        cat = parts[0].strip()
        sub = parts[1].strip()
    else:
        cat, sub = clean_name.strip(), ""
    return cat, sub

def discover_files_by_category(data_dir: str) -> Dict[str, str]:
    """
    Use os.scandir to robustly find CSVs (case-insensitive), regardless of spaces/quotes.
    Returns: { fam_key: full_path }
    fam_key = "Category::Subcategory" or "Category" if no subcategory.
    """
    out: Dict[str, str] = {}
    with os.scandir(data_dir) as it:
        for entry in it:
            if not entry.is_file():
                continue
            # Case-insensitive .csv check (does not care about surrounding quotes)
            if not entry.name.lower().endswith(".csv"):
                continue

            # Build fam key from a cleaned *display* name, not the path
            cleaned = _clean_display_name(os.path.splitext(entry.name)[0])
            cat, sub = _parse_cat_subcat_from_display_name(cleaned)
            fam_key = f"{cat}::{sub}" if sub else cat

            # Keep the true filesystem path exactly as-is for reading
            out[fam_key] = entry.path
    return out


In [ ]:
import numpy as np
import pandas as pd
from typing import Dict, List, Optional
from sklearn.preprocessing import StandardScaler, RobustScaler

def _find_label_column(df: pd.DataFrame) -> str:
    for cand in ["Label", "label", "labels", "y", "target", "class", "Class"]:
        if cand in df.columns:
            return cand
    # last-column fallback if it looks binary
    last = df.columns[-1]
    if set(pd.unique(df[last]).tolist()).issubset({0, 1}):
        return last
    raise RuntimeError("Could not find binary label column (0/1).")

def _split_fam_key(fam_key: str) -> Tuple[str, str]:
    # fam_key is either "Category::Subcategory" or "Category"
    if "::" in fam_key:
        a, b = fam_key.split("::", 1)
        return a.strip(), b.strip()
    return fam_key.strip(), ""

class DataCache:
    def __init__(self, files: Dict[str, str], max_rows: Optional[int] = None):
        self.files = files                      # fam_key -> path
        self.max_rows = max_rows
        self.X: Dict[str, np.ndarray] = {}      # fam_key -> X
        self.y: Dict[str, np.ndarray] = {}      # fam_key -> y
        self.splits = None
        self.scaler = None

        # --- NEW: metadata (does not affect any existing behavior) ---
        self.meta: Dict[str, Dict[str, str]] = {}
        for fam in files.keys():
            cat, sub = _split_fam_key(fam)
            self.meta[fam] = {"category": cat, "subcategory": sub}

    def load_all(self):
        for fam, path in self.files.items():
            df = pd.read_csv(path, nrows=None if self.max_rows is None else int(self.max_rows))
            y_col = _find_label_column(df)
            df.replace([np.inf, -np.inf], np.nan, inplace=True)
            df.dropna(axis=0, how="any", inplace=True)
            df = df[df[y_col].isin([0, 1])]
            y = df[y_col].astype(np.int32).to_numpy()
            X = df.drop(columns=[y_col]).astype(np.float32).to_numpy()
            finite_mask = np.isfinite(X).all(axis=1)
            X, y = X[finite_mask], y[finite_mask]
            self.X[fam] = X
            self.y[fam] = y

    def build_splits(self, rng: np.random.Generator, train_frac=2/3, val_frac=1/6, test_frac=1/6):
        splits = {}
        for fam in self.files.keys():
            y = self.y[fam]
            famspl = {0: {}, 1: {}}
            for c in (0, 1):
                idx = np.where(y == c)[0].copy()
                rng.shuffle(idx)
                n = idx.size
                n_tr = int(round(n * train_frac))
                n_va = int(round(n * val_frac))
                famspl[c]['train'] = idx[:n_tr]
                famspl[c]['val']   = idx[n_tr:n_tr+n_va]
                famspl[c]['test']  = idx[n_tr+n_va:]
            splits[fam] = famspl
        self.splits = splits

    def count(self, fam: str, cls: int, part: str) -> int:
        return int(self.splits[fam][cls][part].size)

    def families_with(self, cls: int, part: str) -> List[str]:
        return [f for f in self.files.keys() if self.count(f, cls, part) > 0]

    def sample(self, fam: str, cls: int, part: str, k: int, rng: np.random.Generator) -> np.ndarray:
        idx = self.splits[fam][cls][part]
        if idx.size == 0:
            D = self.X[next(iter(self.X))].shape[1]
            return np.empty((0, D), np.float32)
        sel = idx if k >= idx.size else rng.choice(idx, size=int(k), replace=False)
        return self.X[fam][sel].astype(np.float32, copy=False)

    def fit_scaler_on_train(self, rng, use_robust=True):
        fams = list(self.files.keys())
        X_train_chunks = []
        for fam in fams:
            idx_pos = self.splits[fam][1]['train']
            idx_neg = self.splits[fam][0]['train']
            for idx in (idx_pos, idx_neg):
                if idx.size == 0:
                    continue
                take = min(10000, idx.size)
                sel = idx if idx.size <= take else rng.choice(idx, take, replace=False)
                X_train_chunks.append(self.X[fam][sel])
        if not X_train_chunks:
            self.scaler = None
            return
        X_fit = np.vstack(X_train_chunks)
        self.scaler = RobustScaler(quantile_range=(5, 95)) if use_robust else StandardScaler()
        self.scaler.fit(X_fit)

    def apply_scaler_all(self):
        if self.scaler is None:
            return
        for fam in self.files.keys():
            self.X[fam] = self.scaler.transform(self.X[fam]).astype(np.float32, copy=False)

    # ---- convenience (optional) ----
    def get_categories(self) -> Dict[str, List[str]]:
        """
        category -> list of subcategories ('' if none).
        """
        out: Dict[str, List[str]] = {}
        for fam, m in self.meta.items():
            out.setdefault(m["category"], []).append(m["subcategory"])
        return out


In [ ]:

    # ---- SNAPSHOT PANELS (reuse your plot_id_ood_separation) ----
def snapshot_sep(cache, encoder, clf, odin, rng, episode: int, label: str,
                part="val", cap_per_family=4000, bins=60):
      """
      Saves 3-panel (hist/ROC/PR) for the given checkpoint label.
      Files go to: plots/ep{episode:02d}_{label}/sep_latest.png
      """
      import os
      out_dir = os.path.join("plots", f"ep{episode:02d}_{label}")
      os.makedirs(out_dir, exist_ok=True)
      return plot_id_ood_separation(
          cache, encoder, clf, odin, rng=rng,
          part=part, families=None, cap_per_family=cap_per_family,
          bins=bins, episode=None, save_dir=out_dir, show=False
      )

In [ ]:
    # ---- TRAJECTORY RECORDER ----
class StageTrajectory:
      def __init__(self, episode: int, save_dir="plots"):
          self.episode = episode
          self.save_dir = save_dir
          self.inner_auc, self.inner_ap = [], []
          self.outer_auc, self.outer_ap = [], []

      def eval_and_push(self, cache, encoder, clf, odin, rng, stage: str):
          res = plot_id_ood_separation(
              cache, encoder, clf, odin, rng=rng,
              part="val", cap_per_family=2000, bins=60,
              episode=None, save_dir=None, show=False
          )
          if stage == "inner":
              self.inner_auc.append(res["roc_auc"]); self.inner_ap.append(res["ap"])
          else:
              self.outer_auc.append(res["roc_auc"]); self.outer_ap.append(res["ap"])

      def save_plot(self):
          import os, matplotlib.pyplot as plt
          os.makedirs(self.save_dir, exist_ok=True)
          fig = plt.figure(figsize=(9, 6))
          ax = fig.add_subplot(2,1,1)
          ax.plot(self.inner_auc + self.outer_auc, lw=2)
          ax.axvline(len(self.inner_auc)-1, ls="--", lw=1)
          ax.set_title(f"Episode {self.episode:02d} • AUC-ROC trajectory")
          ax.set_xlabel("Step"); ax.set_ylabel("AUC-ROC")
          ax2 = fig.add_subplot(2,1,2)
          ax2.plot(self.inner_ap + self.outer_ap, lw=2)
          ax2.axvline(len(self.inner_ap)-1, ls="--", lw=1)
          ax2.set_title("AP trajectory"); ax2.set_xlabel("Step"); ax2.set_ylabel("AP")
          fig.tight_layout()
          fig.savefig(os.path.join(self.save_dir, f"ep{self.episode:02d}_trajectory.png"),
                      dpi=150, bbox_inches="tight")
          plt.close(fig)

In [ ]:

# ---- ODIN TELEMETRY ----
class OdinTelemetry:
    def __init__(self, episode: int, save_dir="plots"):
        self.episode = episode
        self.save_dir = save_dir
        self.rows = []  # dicts with step, T, eps, g_logT, g_logeps, loss

    def push(self, step, odin, g_logT=None, g_logeps=None, loss=None, device="cpu"):
        self.rows.append({
            "step": int(step),
            "T": float(odin.T.detach().to("cpu")),
            "eps": float(odin.eps.detach().to("cpu")),
            "g_logT": float(0.0 if g_logT is None else g_logT),
            "g_logeps": float(0.0 if g_logeps is None else g_logeps),
            "loss": float("nan" if loss is None else float(loss))
        })

    def save_csv_and_plot(self):
        import os, pandas as pd, matplotlib.pyplot as plt
        os.makedirs(self.save_dir, exist_ok=True)
        path = os.path.join(self.save_dir, f"ep{self.episode:02d}_odin_telemetry.csv")
        df = pd.DataFrame(self.rows)
        df.to_csv(path, index=False)

        fig, ax = plt.subplots(2,1, figsize=(9,6), sharex=True)
        ax[0].plot(df["step"], df["T"], lw=2, label="T")
        ax[0].plot(df["step"], df["eps"], lw=2, label="epsilon")
        ax[0].set_ylabel("Value"); ax[0].legend()

        ax[1].plot(df["step"], df["g_logT"], lw=1.5, label="||∇logT||")
        ax[1].plot(df["step"], df["g_logeps"], lw=1.5, label="||∇logε||")
        ax[1].set_xlabel("Outer step"); ax[1].set_ylabel("Grad norm"); ax[1].legend()

        fig.tight_layout()
        fig.savefig(os.path.join(self.save_dir, f"ep{self.episode:02d}_odin_telemetry.png"),
                    dpi=150, bbox_inches="tight")
        plt.close(fig)

In [ ]:
  # ---- FAMILY MARGIN ----
def compute_family_margin(cache, encoder, clf, odin, rng, part="val", cap=2000):
    import numpy as np, torch
    fams = sorted(cache.files.keys())
    out = {}
    for fam in fams:
        m = min(cache.count(fam,1,part), cache.count(fam,0,part), cap)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng).astype(np.float32)
        Xn = cache.sample(fam, 0, part, m, rng).astype(np.float32)
        with torch.no_grad():
            Sa = odin_normal_logit(torch.from_numpy(Xa).to(device), encoder, clf, odin,
                                   steps=VAL_PGD_STEPS).detach().cpu().numpy()
            Sn = odin_normal_logit(torch.from_numpy(Xn).to(device), encoder, clf, odin,
                                   steps=VAL_PGD_STEPS).detach().cpu().numpy()
        out[fam] = float(Sn.mean() - Sa.mean())
    return out

def plot_family_margin(m_pre: dict, m_inner: dict, m_outer: dict,
                       episode: int, m_target: float, save_dir="plots"):
    import os, matplotlib.pyplot as plt, numpy as np
    os.makedirs(save_dir, exist_ok=True)
    fams = sorted(set(m_pre) | set(m_inner) | set(m_outer))
    x = np.arange(len(fams)); w = 0.25
    fig, ax = plt.subplots(figsize=(max(10, len(fams)*0.6), 4.5))
    ax.bar(x - w, [m_pre.get(f, np.nan) for f in fams], width=w, label="pre")
    ax.bar(x      , [m_inner.get(f, np.nan) for f in fams], width=w, label="after_inner")
    ax.bar(x + w, [m_outer.get(f, np.nan) for f in fams], width=w, label="after_outer")
    ax.axhline(m_target, ls="--", lw=1, color="k", label="margin m")
    ax.set_xticks(x); ax.set_xticklabels(fams, rotation=45, ha="right")
    ax.set_ylabel("Mean calibrated logit gap (ID − OOD)")
    ax.set_title(f"Episode {episode:02d} • per-family margin")
    ax.legend()
    fig.tight_layout()
    fig.savefig(os.path.join(save_dir, f"ep{episode:02d}_family_margin.png"),
                dpi=150, bbox_inches="tight")
    plt.close(fig)

In [ ]:
# ============================ helpers: per-family eval ============================

def build_perfam_eval_pools(cache, fams, rng, part: str, cap_per_family: int):
    """Return {fam: (X, y)} with balanced ID(0)/OOD(1) blocks per family from split `part`."""
    def _balanced_block(fam):
        m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap_per_family)
        if m <= 0:
            D = cache.X[fam].shape[1]
            return np.empty((0, D), np.float32), np.empty((0,), np.int64)
        Xa = cache.sample(fam, 1, part, m, rng)  # OOD(+1)
        Xn = cache.sample(fam, 0, part, m, rng)  # ID(0)
        X  = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        y  = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
        return X, y

    pools = {}
    for fam in fams:
        Xf, yf = _balanced_block(fam)
        if Xf.size > 0:
            pools[fam] = (Xf, yf)
    return pools


def score_anomaly_numpy(X, encoder, clf, odin, steps: int = 1):
    """Anomaly score = 1 - p_norm via your ODIN routine (no tau argument)."""
    p_norm = odin_pnorm_numpy(X, encoder, clf, odin, steps=steps)
    return 1.0 - p_norm


def eval_stage_per_family(eval_pools, encoder, clf, odin, steps: int, stage_name: str):
    """
    Compute per-family anomaly scores for a given stage.
    Returns: { fam: {'y': y, 'scores': s} }
    """
    out = {}
    for fam, (Xf, yf) in eval_pools.items():
        s = score_anomaly_numpy(Xf, encoder, clf, odin, steps=steps)
        out[fam] = {"y": yf, "scores": s, "stage": stage_name}
    return out

def plot_per_family_overlays(
    results_pre,
    results_inner,
    results_outer,
    episode: int,
    save_root: str,
    bins: int = 60,
    verbose: bool = False,
):
    """
    None-safe wrapper:
      - Accepts None for any of results_pre/inner/outer.
      - Computes the union of families present in any available result dict.
      - Skips cleanly if nothing to plot.
    """
    import os
    import numpy as np
    import matplotlib.pyplot as plt

    # Coerce None -> empty dict
    results_pre   = results_pre   or {}
    results_inner = results_inner or {}
    results_outer = results_outer or {}

    fams = sorted(set(results_pre.keys()) | set(results_inner.keys()) | set(results_outer.keys()))
    save_dir = os.path.join(save_root, f"ep{episode:02d}")
    os.makedirs(save_dir, exist_ok=True)

    if not fams:
        print(f"[PLOT] No per-family results to plot for episode {episode:02d}; skip.")
        return

    saved = []
    for fam in fams:
        # Each results_* dict is expected to have something like:
        #   results_*[fam] = {"id_scores": np.array([...]), "ood_scores": np.array([...])}
        fam_pre   = results_pre.get(fam,   None)
        fam_inner = results_inner.get(fam, None)
        fam_outer = results_outer.get(fam, None)

        # Build list of (label, array) for overlay
        layers = []
        if fam_pre is not None:
            for tag, arr in fam_pre.items():
                if arr is not None and np.size(arr) > 0:
                    layers.append((f"pre/{tag}", np.asarray(arr)))
        if fam_inner is not None:
            for tag, arr in fam_inner.items():
                if arr is not None and np.size(arr) > 0:
                    layers.append((f"inner/{tag}", np.asarray(arr)))
        if fam_outer is not None:
            for tag, arr in fam_outer.items():
                if arr is not None and np.size(arr) > 0:
                    layers.append((f"outer/{tag}", np.asarray(arr)))

        if not layers:
            if verbose:
                print(f"[PLOT] Family '{fam}': no arrays to plot.")
            continue

        # Draw overlayed histograms
        plt.figure(figsize=(6, 4), dpi=140)
        for lbl, arr in layers:
            try:
                plt.hist(arr, bins=bins, density=True, histtype="step", linewidth=1.2, label=lbl)
            except Exception as e:
                if verbose:
                    print(f"[PLOT] Family '{fam}' layer '{lbl}' failed: {e}")

        plt.title(f"Episode {episode:02d} — {fam}")
        plt.xlabel("score"); plt.ylabel("density"); plt.legend(frameon=False)
        plt.grid(alpha=0.25, linewidth=0.6)
        out = os.path.join(save_dir, f"{fam.replace(' ', '_').replace(':','-')}.png")
        plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close()
        saved.append(out)

    print(f"[PLOT] Per-family overlays saved ({len(saved)} figs) → {save_dir}")


In [ ]:


# ------------------------ Models ------------------------
class ResBlock(nn.Module):
    def __init__(self, d, drop_path=0.0):
        super().__init__()
        self.ln1 = nn.LayerNorm(d)
        self.fc1 = nn.Linear(d, d)
        self.ln2 = nn.LayerNorm(d)
        self.fc2 = nn.Linear(d, d)
        self.drop_path = float(drop_path)
    def forward(self, x):
        h = self.fc1(F.gelu(self.ln1(x)))
        h = self.fc2(F.gelu(self.ln2(h)))
        if self.training and self.drop_path>0 and torch.rand(()) < self.drop_path:
            return x
        return x + 0.5*h

class Encoder(nn.Module):
    def __init__(self, in_dim, hid=256, depth=3, p_drop=0.1, bottleneck=128, feat_drop=0.0):
        super().__init__()
        self.inp = nn.Linear(in_dim, hid)
        self.feat_drop = feat_drop
        self.blocks = nn.ModuleList([ResBlock(hid, drop_path=0.05) for _ in range(depth)])
        self.dropout = nn.Dropout(p_drop)
        self.out = nn.Linear(hid, bottleneck)
    def forward(self, x):
        if self.training and self.feat_drop>0:
            m = torch.rand_like(x) > self.feat_drop
            x = x * m
        h = F.gelu(self.inp(x))
        for blk in self.blocks: h = blk(h)
        h = self.dropout(h)
        return self.out(h)

class Decoder(nn.Module):
    def __init__(self, bottleneck=128, hid=256, out_dim=77, depth=2):
        super().__init__()
        self.fc = nn.Linear(bottleneck, hid)
        self.blocks = nn.ModuleList([ResBlock(hid) for _ in range(depth)])
        self.out = nn.Linear(hid, out_dim)
    def forward(self, z):
        h = F.gelu(self.fc(z))
        for blk in self.blocks: h = blk(h)
        return self.out(h)

class Classifier(nn.Module):
    def __init__(self, bottleneck=128, num_classes=2):
        super().__init__()
        self.ln = nn.LayerNorm(bottleneck)
        self.fc = nn.Linear(bottleneck, num_classes)
    def forward(self, z):
        return self.fc(F.gelu(self.ln(z)))

In [ ]:
# ===== ODIN: epsilon-fix (replace the old pieces) ============================
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

# ===== ODIN params (bounded, learnable; accepts T_bounds / eps_bounds) =====
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

class ODINParams(nn.Module):
    """
    Learnable temperature T and perturbation size eps, both bounded.
    Supports:
        - ODINParams(T0=..., eps0=..., learn_T=..., learn_eps=...,
                     T_bounds=(Tmin, Tmax), eps_bounds=(Emin, Emax))
    Access:
        - odin.T        -> scalar tensor on the right device
        - odin.eps      -> scalar tensor on the right device
        - odin.get_T()  -> same as .T (helper if you prefer a function)
        - odin.get_eps()-> same as .eps
    """
    def __init__(
        self,
        T0: float,
        eps0: float,
        learn_T: bool = True,
        learn_eps: bool = True,
        T_bounds = (0.5, 10.0),
        eps_bounds = (1e-5, 2e-2),
    ):
        super().__init__()
        # store bounds
        self.T_min, self.T_max   = [float(x) for x in T_bounds]
        self.E_min, self.E_max   = [float(x) for x in eps_bounds]

        # --- helpers to map (a,b) -> R and back ---
        def inv_sigmoid_affine(x, a, b):
            # map x in (a,b) -> raw in R
            x = float(np.clip((x - a) / (b - a), 1e-6, 1 - 1e-6))
            return np.log(x) - np.log(1 - x)

        # T: parameterize with sigmoid onto [T_min, T_max]
        T_raw0 = inv_sigmoid_affine(T0, self.T_min, self.T_max)
        self._T_raw  = nn.Parameter(torch.tensor(T_raw0, dtype=torch.float32),
                                    requires_grad=bool(learn_T))

        # eps: parameterize with softplus (+E_min) and clamp to [E_min, E_max]
        # invert softplus approximately: rho s.t. softplus(rho)+E_min ≈ eps0
        rho0 = np.log(np.exp(max(eps0 - self.E_min, 1e-12)) - 1.0)
        self._eps_raw = nn.Parameter(torch.tensor(rho0, dtype=torch.float32),
                                     requires_grad=bool(learn_eps))

    # ---- public API: properties return scalar tensors on the module's device ----
    @property
    def T(self) -> torch.Tensor:
        # affine-sigmoid to [T_min, T_max]
        return self.T_min + (self.T_max - self.T_min) * torch.sigmoid(self._T_raw)

    @property
    def eps(self) -> torch.Tensor:
        # softplus to (0, inf), shift by E_min, then clamp to [E_min, E_max]
        e = F.softplus(self._eps_raw) + self.E_min
        return torch.clamp(e, self.E_min, self.E_max)

    # ---- optional helpers if you prefer function-style reads ----
    def get_T(self) -> torch.Tensor:
        return self.T

    def get_eps(self) -> torch.Tensor:
        return self.eps


def _odin_direction(x, encoder, clf, Tcur: torch.Tensor) -> torch.Tensor:
    """
    Build the ODIN/FGSM direction ∂/∂x of -log p_top. We only use its sign
    and we *detach the sign* so the direction is treated as a constant in the
    subsequent update (STE-style).
    """
    x_req = x.detach().clone().requires_grad_(True)
    logits = clf(encoder(x_req)) / Tcur
    p = logits.softmax(dim=1)
    top = p.max(dim=1).values
    loss = -torch.log(top.clamp_min(1e-12)).mean()
    g = torch.autograd.grad(loss, x_req, only_inputs=True)[0]
    return g.sign().detach()  # <-- direction is constant (no grads through sign)


def odin_perturb(x, encoder, clf, odin: ODINParams, steps: int = 1, tau: float = 0.0):
    """
    Multi-step ODIN where the *update* is differentiable w.r.t. eps/T.
    Only the direction is detached. No no_grad() around the update.
    If steps<=0, returns x unchanged.
    """
    if steps <= 0:
        return x

    Tcur = odin.T          # stays in-graph so T can learn if required
    eps  = odin.eps        # stays in-graph so eps can learn
    step = eps / max(1, steps)

    x0 = x
    x_adv = x
    for _ in range(steps):
        d = _odin_direction(x_adv, encoder, clf, Tcur)  # detached sign
        x_adv = x_adv + step * d                        # <-- differentiable w.r.t. eps/T
        if tau > 0.0:
            x_adv = (1.0 - tau) * x_adv + tau * x0     # optional EMA pullback

    return x_adv


def odin_forward_train(x: torch.Tensor, encoder, clf, odin: ODINParams, *, steps: int = 1):
    """
    Training forward:
      - builds x_adv with ODIN (update in-graph),
      - returns calibrated logits on x_adv.
    """
    if steps <= 0:
        z = encoder(x)
        return clf(z) / odin.T

    x_adv = odin_perturb(x, encoder, clf, odin, steps=steps)
    z = encoder(x_adv)
    return clf(z) / odin.T


def odin_score_eval(x: torch.Tensor,
                    encoder: torch.nn.Module,
                    clf: torch.nn.Module,
                    odin,                      # has .T and .eps tensors
                    steps: int = 1) -> torch.Tensor:
    """
    Returns p_norm in [0,1] for each sample in x using ODIN perturbation.
    Works even when called from a @torch.no_grad() wrapper by locally
    enabling autograd only for the perturbation step.
    """
    device = next(clf.parameters()).device
    x = x.to(device)

    # current temperature & step
    Tcur = (odin.T if isinstance(odin.T, torch.Tensor) else torch.as_tensor(odin.T, device=device)).to(device)
    step = (odin.eps if isinstance(odin.eps, torch.Tensor) else torch.as_tensor(odin.eps, device=device)).to(device)

    # make a leaf that tracks grad
    x_req = x.detach().clone().requires_grad_(True)

    # one or more ODIN steps
    for _ in range(max(1, int(steps))):
        with torch.enable_grad():
            logits0 = clf(encoder(x_req)) / Tcur
            # pick confidence of the predicted class; maximize it to get sign of grad
            probs  = logits0.softmax(dim=1) if logits0.shape[-1] > 1 else torch.sigmoid(logits0)
            # take the max confidence across classes (binary or multi)
            conf_max = probs.max(dim=1).values if probs.ndim == 2 else probs.squeeze(-1)
            loss = -conf_max.mean()  # ascend confidence
            g = torch.autograd.grad(loss, x_req, retain_graph=False, create_graph=False, allow_unused=False)[0]
            d = g.sign()

        # take a small step and re-leaf
        x_req = (x_req + step * d).detach().requires_grad_(True)

    # final forward for the calibrated normal probability
    with torch.no_grad():
        logits = clf(encoder(x_req)) / Tcur
        if logits.shape[-1] == 1:
            # binary logit: assume logit for "normal" is the single channel
            p_norm = torch.sigmoid(logits.squeeze(-1))
        else:
            # multi-class: assume class index 0 == normal
            p_norm = logits.softmax(dim=1)[:, 0]

    return p_norm  # [B] in [0,1]




def odin_normal_logit(x, encoder, clf, odin: ODINParams, *, steps=1):
    """
    EVAL-ONLY scorer: returns (calibrated) normal-class logit after 1-step ODIN.
    Side-effect free (no graph to params or eps).
    """
    assert steps >= 0
    if steps == 0:
        with torch.set_grad_enabled(False):
            z = encoder(x)
            logits = clf(z) / odin.T            # <-- calibrate
            return logits[:, 0]

    # 1-step FGSM direction (builds a tiny graph only to get input grads)
    x_adv = x.detach().clone().requires_grad_(True)
    z0 = encoder(x_adv)
    logits0 = clf(z0) / odin.T                  # <-- calibrate in the targeting step too
    logit_norm0 = logits0[:, 0]
    loss = -logit_norm0.sum()
    grads = torch.autograd.grad(loss, x_adv, only_inputs=True)[0]

    # update adversarial input WITHOUT building a graph to eps/params
    with torch.no_grad():
        x_adv = x_adv + odin.eps * _sign(grads)

    # final forward for the score (no grad)
    with torch.set_grad_enabled(False):
        z = encoder(x_adv)
        logits = clf(z) / odin.T                # <-- calibrate
        return logits[:, 0]


In [ ]:

# metrics.py
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score


def compute_micro_macro(y_true, y_score, y_pred, fam_ids):
    """
    y_true: (N,) 0/1
    y_score: (N,) anomaly score or probability for ROC/AP
    y_pred: (N,) 0/1 at your chosen threshold
    fam_ids: (N,) int ids per sample
    """
    # ---- MICRO (pooled)
    prec_mi, rec_mi, f1_mi, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
    auc_mi = roc_auc_score(y_true, y_score)
    ap_mi  = average_precision_score(y_true, y_score)

    # ---- MACRO (avg of per-family)
    fams = np.unique(fam_ids)
    precs, recs, f1s, aucs, aps = [], [], [], [], []
    for f in fams:
        m = (fam_ids == f)
        if m.sum() < 2 or len(np.unique(y_true[m])) < 2:
            continue
        p_f, r_f, f_f, _ = precision_recall_fscore_support(y_true[m], y_pred[m], average='binary', zero_division=0)
        precs.append(p_f); recs.append(r_f); f1s.append(f_f)
        aucs.append(roc_auc_score(y_true[m], y_score[m]))
        aps.append(average_precision_score(y_true[m], y_score[m]))
    macro = dict(
        precision=np.mean(precs) if precs else 0.0,
        recall=np.mean(recs) if recs else 0.0,
        f1=np.mean(f1s) if f1s else 0.0,
        auc=np.mean(aucs) if aucs else 0.0,
        ap=np.mean(aps) if aps else 0.0,
    )
    micro = dict(precision=prec_mi, recall=rec_mi, f1=f1_mi, auc=auc_mi, ap=ap_mi)
    return micro, macro

## ======================== Utilities used by the trainers ========================
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# assumes these exist in your file:
# - device
# - NUM_WORKERS, PIN_MEMORY
# - INNER_LR_MAX, WEIGHT_DECAY, CLIP_NORM, USE_AMP, LAMBDA_REC
# - OUTER_MARGIN_M, OUTER_MARGIN_ALPHA
# - OUTER_BATCH, OUTER_EPOCHS, TRAIN_PGD_STEPS, ODIN_TAU
# - odin_normal_logit(x, encoder, clf, odin, steps, tau) -> [B,1] logits on "normal" class

def _safe_grad_norm(p: torch.nn.Parameter) -> float:
    g = getattr(p, "grad", None)
    if g is None:
        return 0.0
    if not torch.isfinite(g).all():
        return 0.0
    return float(g.detach().norm().item())

In [ ]:
# ------------------------ Inner Trainer (with benign-center EMA) ------------------------
# ======================== Utilities used by the trainers ========================
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# expects in your file:
# - device
# - NUM_WORKERS, PIN_MEMORY
# - INNER_LR_MAX, WEIGHT_DECAY, CLIP_NORM, USE_AMP, LAMBDA_REC
# - OUTER_MARGIN_M, OUTER_MARGIN_ALPHA
# - OUTER_BATCH, OUTER_EPOCHS, TRAIN_PGD_STEPS, ODIN_TAU
# - odin_normal_logit(x, encoder, clf, odin, steps, tau) -> [B,1] normal-class logits

def _safe_grad_norm(p: torch.nn.Parameter) -> float:
    g = getattr(p, "grad", None)
    if g is None or not torch.isfinite(g).all():
        return 0.0
    return float(g.detach().norm().item())

In [ ]:


# ------------------------ Inner Trainer (with benign-center EMA) ------------------------
# ------------------------ Outer Trainer (hardened against NaNs) ------------------------
# ------------------------ Inner Trainer (unchanged) ------------------------
class InnerTrainer:
    def __init__(self, encoder: nn.Module, decoder: nn.Module, clf: nn.Module,
                 lr_max: float = INNER_LR_MAX, weight_decay: float = WEIGHT_DECAY,
                 clip_norm: float = CLIP_NORM, use_amp: bool = USE_AMP,
                 lambda_rec: float = LAMBDA_REC, total_inner_steps_est: int | None = None):
        self.encoder, self.decoder, self.clf = encoder, decoder, clf
        self.clip_norm = clip_norm
        self.use_amp = use_amp and (device.type == "cuda")
        self.lambda_rec = lambda_rec
        params = list(self.encoder.parameters()) + list(self.decoder.parameters()) + list(self.clf.parameters())
        for p in params: p.requires_grad_(True)
        self.opt = torch.optim.AdamW(params, lr=lr_max, weight_decay=weight_decay, betas=(0.9, 0.99))
        if total_inner_steps_est is None:
            total_inner_steps_est = 50_000
        self.sched = torch.optim.lr_scheduler.CosineAnnealingLR(self.opt, T_max=total_inner_steps_est)
        self.scaler = torch.amp.GradScaler('cuda', enabled=self.use_amp)

    def train_one_episode(self, meta_tasks: list, batch_size: int = INNER_BATCH, epochs: int = INNER_EPOCHS):
        # enable grads
        for p in self.encoder.parameters(): p.requires_grad_(True)
        for p in self.decoder.parameters(): p.requires_grad_(True)
        for p in self.clf.parameters():     p.requires_grad_(True)
        self.encoder.train(); self.decoder.train(); self.clf.train()
        device_ = next(self.encoder.parameters()).device

        # build loaders per task
        task_loaders = []
        for (Xi, yi, _) in meta_tasks:
            Xi_t = torch.from_numpy(Xi.astype(np.float32, copy=False))
            yi_t = torch.from_numpy(yi.astype(np.int64,  copy=False))
            ds = TensorDataset(Xi_t, yi_t)
            dl = DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=False,
                            num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
            task_loaders.append(dl)
        if len(task_loaders) == 0:
            print("[INNER/BCE] No data in meta_tasks; skipping.")
            return

        steps_per_epoch = max(1, max(len(dl) for dl in task_loaders))
        def _fresh_iters(): return [iter(dl) for dl in task_loaders]
        iters = _fresh_iters()
        params = list(self.encoder.parameters()) + list(self.decoder.parameters()) + list(self.clf.parameters())

        for ep in range(1, epochs + 1):
            epoch_losses = []
            for _ in range(steps_per_epoch):
                batches = []
                for ti, it in enumerate(iters):
                    try:
                        xb, yb = next(it)
                    except StopIteration:
                        iters[ti] = iter(task_loaders[ti])
                        try:
                            xb, yb = next(iters[ti])
                        except StopIteration:
                            continue
                    batches.append((xb.to(device_).float(), yb.to(device_).long()))
                if len(batches) == 0:
                    continue

                xb = torch.cat([b[0] for b in batches], dim=0)
                yb = torch.cat([b[1] for b in batches], dim=0)
                tgt = (yb == 0).float()  # normal=1, anomaly=0 → target for normal-logit

                self.opt.zero_grad(set_to_none=True)
                with torch.amp.autocast('cuda', enabled=self.use_amp):
                    z = self.encoder(xb)
                    logits = self.clf(z)            # [B, 2]
                    logit_norm = logits[:, 0]       # normal-class logit
                    bce = F.binary_cross_entropy_with_logits(logit_norm, tgt)
                    loss = bce
                    if self.lambda_rec > 0:
                        xhat = self.decoder(z)
                        loss = loss + self.lambda_rec * F.mse_loss(xhat, xb)

                self.scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(params, max_norm=self.clip_norm)
                self.scaler.step(self.opt)
                self.scaler.update()
                self.sched.step()

                epoch_losses.append(float(loss.detach().item()))
            mean_loss = np.mean(epoch_losses) if epoch_losses else 0.0
            print(f"[INNER/BCE] Ep{ep:02d} | Loss={mean_loss:.5f} | lr={self.opt.param_groups[0]['lr']:.2e}")



In [ ]:
# class OuterTrainer(torch.nn.Module):
#     def __init__(
#         self,
#         encoder: nn.Module,
#         clf: nn.Module,
#         odin: ODINParams,
#         lr_odin: float = 1e-3,
#         wd_odin: float = 0.0,
#         amp: bool = True,
#         loss_mode: str = "pair_auc",   # "bce", "margin", or "pair_auc"
#         pair_margin: float = 0.5,
#         pair_max: float | None = None,  # <--- NEW: accepted but not used
#     ):
#         super().__init__()
#         self.encoder = encoder
#         self.clf     = clf
#         self.odin    = odin
#         self.loss_mode   = loss_mode
#         self.pair_margin = pair_margin
#         self.pair_max    = pair_max   # you can ignore this later

#         # ---- freeze backbone (φ) for outer loop ----
#         for p in self.encoder.parameters():
#             p.requires_grad_(False)
#         for p in self.clf.parameters():
#             p.requires_grad_(False)

#         # ---- optimizer: ONLY ODIN params that require_grad ----
#         odin_params = [p for p in self.odin.parameters() if p.requires_grad]
#         if odin_params and lr_odin > 0.0:
#             self.optim_odin = torch.optim.Adam(odin_params, lr=lr_odin, weight_decay=wd_odin)
#             self.scaler     = torch.cuda.amp.GradScaler(enabled=amp)
#         else:
#             self.optim_odin = None
#             self.scaler     = None
#         self.amp = amp

#     # ---------- helpers: expose T and eps ----------
#     def _T(self) -> torch.Tensor:
#         """Learnable temperature (in-graph)."""
#         T = self.odin.T
#         if not isinstance(T, torch.Tensor):
#             T = torch.as_tensor(T, device=next(self.clf.parameters()).device)
#         return T

#     def _eps(self) -> torch.Tensor:
#         """
#         FIXED epsilon: treated as a constant in the graph.
#         Even if ODINParams stores it as a tensor, we detach it here so
#         outer gradients NEVER flow into ε.
#         """
#         eps = self.odin.eps
#         dev = next(self.clf.parameters()).device
#         if not isinstance(eps, torch.Tensor):
#             eps = torch.as_tensor(eps, device=dev)
#         return eps.detach()   # <--- key line: epsilon is constant for outer loss

#     # ---------- gradient direction for ODIN FGSM ----------
#     def _adv_dir(self, xb: torch.Tensor) -> torch.Tensor:
#         """
#         Compute normalized gradient direction of -log p_norm
#         (standard ODIN direction). T is detached here so we only
#         use it as a scalar, not for gradients from this part.
#         """
#         xb = xb.clone().detach().requires_grad_(True)
#         with torch.cuda.amp.autocast(enabled=self.amp):
#             logits = self.clf(self.encoder(xb)) / self._T().detach()
#             p_norm = torch.softmax(logits, dim=-1)[:, 1]   # normal class = 1
#             loss   = -p_norm.mean()
#         loss.backward()
#         g = xb.grad
#         g = g / (g.view(g.size(0), -1).norm(dim=1, keepdim=True) + 1e-12)
#         return g.detach()


class OuterTrainer(nn.Module):
    def __init__(self, encoder, clf, odin,
                 lr_odin=1e-3, wd_odin=0.0, amp=False,
                 loss_mode="bce", pair_margin=1.0, pair_max=2048,
                 train_clf: bool = False):
        super().__init__()
        self.encoder, self.clf, self.odin = encoder, clf, odin
        self.amp = bool(amp)
        self.loss_mode = loss_mode
        self.pair_margin = float(pair_margin)
        self.pair_max = int(pair_max)
        self.train_clf = bool(train_clf)

        # --- freeze encoder, maybe unfreeze clf, build optimizer ---
        for p in self.encoder.parameters():
            p.requires_grad_(False)

        if self.train_clf:
            for p in self.clf.parameters():
                p.requires_grad_(True)
        else:
            for p in self.clf.parameters():
                p.requires_grad_(False)

        odin_params = [
            p for p in [
                getattr(self.odin, "_T_raw", None),
                getattr(self.odin, "_eps_raw", None),
            ]
            if (p is not None and p.requires_grad)
        ]
        params = list(odin_params)
        if self.train_clf:
            params += [p for p in self.clf.parameters() if p.requires_grad]

        self.optim_odin = (
            None if len(params) == 0
            else torch.optim.Adam(params, lr=lr_odin, weight_decay=wd_odin)
        )
        self.scaler = torch.cuda.amp.GradScaler(
            enabled=self.amp and (self.optim_odin is not None)
        )

    # >>> ADD THESE TWO METHODS <<<
    def _T(self):
        """Current ODIN temperature as positive scalar tensor."""
        if hasattr(self.odin, "_T_raw"):
            return self.odin._T_raw.exp()
        # fallback: just return 1.0 on same device
        dev = next(self.odin.parameters()).device
        return torch.ones((), device=dev)

    def _eps(self):
        """Current ODIN epsilon as positive scalar tensor."""
        if hasattr(self.odin, "_eps_raw"):
            return self.odin._eps_raw.exp()
        dev = next(self.odin.parameters()).device
        return torch.zeros((), device=dev)

    # ... keep your existing train_one_episode(...) below ...

        # ---------- gradient direction for ODIN FGSM ----------
    def _adv_dir(self, xb: torch.Tensor) -> torch.Tensor:
        """
        Compute normalized gradient direction of -log p_norm
        (standard ODIN direction). T is detached here so we only
        use it as a scalar, not for gradients from this part.
        """
        xb = xb.clone().detach().requires_grad_(True)
        with torch.cuda.amp.autocast(enabled=self.amp):
            logits = self.clf(self.encoder(xb)) / self._T().detach()
            p_norm = torch.softmax(logits, dim=-1)[:, 1]   # normal class = 1
            loss   = -p_norm.mean()
        loss.backward()
        g = xb.grad
        g = g / (g.view(g.size(0), -1).norm(dim=1, keepdim=True) + 1e-12)
        return g.detach()


    def train_one_episode(
        self,
        Xq: np.ndarray,
        yq: np.ndarray,
        batch_size: int = 256,
        epochs: int = 5,
        **unused_kwargs,   # <--- swallow steps=... and anything else
    ):
        """
        Outer loop: update ONLY T so that normal vs OOD logits separate better.
        Extra keyword arguments (e.g. steps=...) are accepted and ignored.
        """
        if self.optim_odin is None:
            return

        device = next(self.clf.parameters()).device
        X = torch.from_numpy(Xq).to(device=device, dtype=torch.float32)
        y = torch.from_numpy(yq).to(device=device, dtype=torch.long)

        # split into ID (normal) vs OOD (anomaly)
        id_mask  = (y == 1)  # normal
        ood_mask = (y == 0)  # anomaly
        X_id, X_ood = X[id_mask], X[ood_mask]
        y_id = y[id_mask].float()
        y_ood = y[ood_mask].float()

        T0  = float(self._T().detach().cpu())
        E0  = float(self._eps().detach().cpu())

        for ep in range(1, epochs + 1):
            self.odin.train()
            ep_loss = 0.0
            c_batches = 0

            # simple balanced batches of ID / OOD
            perm_id  = torch.randperm(X_id.size(0), device=device)
            perm_ood = torch.randperm(X_ood.size(0), device=device)
            nb = min(len(perm_id), len(perm_ood)) // (batch_size // 2)
            if nb == 0:
                continue

            for b in range(nb):
                i0 = b * (batch_size // 2)
                i1 = (b + 1) * (batch_size // 2)
                xb = torch.cat([X_id[perm_id[i0:i1]], X_ood[perm_ood[i0:i1]]], dim=0)
                yb = torch.cat([torch.ones(i1 - i0, device=device),
                                torch.zeros(i1 - i0, device=device)], dim=0)

                # build ODIN-perturbed batch with FIXED eps but learnable T
                d     = self._adv_dir(xb)
                x_adv = xb + self._eps() * d
                x_adv = x_adv.clamp_(-10.0, 10.0)  # just safety

                with torch.cuda.amp.autocast(enabled=self.amp):
                    logits = self.clf(self.encoder(x_adv)) / self._T()
                    # normal-class logit
                    z_norm = logits[:, 1]

                    if self.loss_mode == "bce":
                        # BCE on logits (target: 1 for ID, 0 for OOD)
                        loss = F.binary_cross_entropy_with_logits(
                            z_norm, yb.float()
                        )

                    elif self.loss_mode == "margin":
                        # encourage z_norm(ID) - z_norm(OOD) >= M
                        z_id  = z_norm[yb == 1]
                        z_ood = z_norm[yb == 0]
                        if (z_id.numel() > 0) and (z_ood.numel() > 0):
                            diff = z_id.mean() - z_ood.mean()
                            loss = F.relu(self.pair_margin - diff)
                        else:
                            loss = torch.zeros((), device=device)

                    elif self.loss_mode == "pair_auc":
                        # hinge-style pairwise AUC: max(0, m - (z_id - z_ood))
                        z_id  = z_norm[yb == 1]
                        z_ood = z_norm[yb == 0]
                        if (z_id.numel() > 0) and (z_ood.numel() > 0):
                            # all pair differences ID - OOD
                            diffs = z_id[:, None] - z_ood[None, :]
                            loss = F.relu(self.pair_margin - diffs).mean()
                        else:
                            loss = torch.zeros((), device=device)

                    else:
                        raise ValueError(f"Unknown loss_mode '{self.loss_mode}'")

                self.optim_odin.zero_grad(set_to_none=True)
                if self.scaler is not None:
                    self.scaler.scale(loss).backward()
                    self.scaler.unscale_(self.optim_odin)
                    torch.nn.utils.clip_grad_norm_(self.odin.parameters(), 10.0)
                    self.scaler.step(self.optim_odin)
                    self.scaler.update()
                else:
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.odin.parameters(), 10.0)
                    self.optim_odin.step()

                ep_loss += float(loss.detach().cpu())
                c_batches += 1

            # logging (and gradient norms)
            gT = float(_safe_grad_norm(getattr(self.odin, "_T_raw", None)))
            gE = float(_safe_grad_norm(getattr(self.odin, "_eps_raw", None)))
            Tcur = float(self._T().detach().cpu())
            Ecur = float(self._eps().detach().cpu())
            lr   = (self.optim_odin.param_groups[0]["lr"]
                    if self.optim_odin is not None else 0.0)
            print(
                f"[OUTER/{self.loss_mode.upper()}] Ep{ep:02d} | "
                f"Loss={ep_loss / max(1, c_batches):.5f} "
                f"T0={T0:.6f}->T={Tcur:.6f}  eps0={E0:.6e}->eps={Ecur:.6e} "
                f"||∇rawT||={(0.0 if math.isnan(gT) else gT):.3e} "
                f"||∇raweps||={(0.0 if math.isnan(gE) else gE):.3e}  lr_odin={lr:.2e}"
            )


In [ ]:



#------- Episode builders ------------------------
def _draw_k_up_to(rng: np.random.Generator, k_avail: int, k_max: int, k_min: int) -> int:
    if k_avail <= 0: return 0
    hi = min(k_max, k_avail)
    if hi < k_min: return 0
    return int(rng.integers(k_min, hi + 1))

def select_inner_fams_up_to(cache, rng, allowed_fams, part, inner_max_fams, inner_min_fams=2):
    cands = [f for f in cache.families_with(cls=1, part=part) if f in allowed_fams]
    rng.shuffle(cands)
    k = _draw_k_up_to(rng, len(cands), inner_max_fams, inner_min_fams)
    return cands[:k]

def select_outer_ood_fams_up_to(cache, rng, inner_used, allowed_fams, part, outer_max_fams, outer_min_fams=1):
    # don't reuse inner families as OOD; rotate randomly for diversity
    cands = [f for f in cache.families_with(cls=1, part=part) if (f in allowed_fams) and (f not in inner_used)]
    rng.shuffle(cands)
    k = _draw_k_up_to(rng, len(cands), outer_max_fams, outer_min_fams)
    return cands[:k]

def _balanced_downsample(X: np.ndarray, y: np.ndarray, target_total: Optional[int], rng: np.random.Generator):
    if (target_total is None) or (target_total <= 0) or (X.shape[0] <= target_total):
        return X, y
    half = target_total // 2
    idx_pos = np.where(y == 1)[0]; idx_neg = np.where(y == 0)[0]
    take_pos = min(len(idx_pos), half); take_neg = min(len(idx_neg), half)
    short = 2*half - (take_pos + take_neg)
    if short > 0:
        if take_pos < half: take_neg = min(len(idx_neg), take_neg + short)
        elif take_neg < half: take_pos = min(len(idx_pos), take_pos + short)
    sel_pos = idx_pos if take_pos >= len(idx_pos) else rng.choice(idx_pos, size=take_pos, replace=False)
    sel_neg = idx_neg if take_neg >= len(idx_neg) else rng.choice(idx_neg, size=take_neg, replace=False)
    sel = np.concatenate([sel_pos, sel_neg]); rng.shuffle(sel)
    X2 = X[sel].astype(np.float32, copy=False); y2 = y[sel].astype(np.int64, copy=False)
    return X2, y2

def build_inner_episode_batch(cache, rng, part: str, fams: List[str], pos_per_fam: int, neg_per_fam):
    X_parts, y_parts, used = [], [], []
    for fam in fams:
        Xa = cache.sample(fam, 1, part, pos_per_fam, rng)
        if Xa.size == 0: continue
        kneg = Xa.shape[0] if (isinstance(neg_per_fam, str) and neg_per_fam == "match") else int(neg_per_fam)
        kneg = max(1, kneg)
        Xn = cache.sample(fam, 0, part, kneg, rng)
        if Xn.size == 0: continue
        X_parts += [Xa, Xn]
        y_parts += [np.ones(Xa.shape[0], np.int64), np.zeros(Xn.shape[0], np.int64)]
        used.append(fam)
    if not X_parts:
        return np.empty((0,0), np.float32), np.empty((0,), np.int64), []
    X = np.vstack(X_parts).astype(np.float32, copy=False)
    y = np.concatenate(y_parts, axis=0)
    X, y = _balanced_downsample(X, y, target_total=INNER_EPISODE_MAX, rng=rng)
    return X, y, used


def build_outer_episode_batch(
    cache,
    rng: np.random.Generator,
    *,
    ood_fams: list,
    part: str = "train",
    per_class_min: int = 100,
    per_class_max: int = 200,
):
    """
    Outer batch builder for meta-ODIN.

    • Uses *all* candidate OOD families in `ood_fams`
      (held-out families never appear here by construction).
    • For each family, samples K ∈ [per_class_min, per_class_max] anomalies (label=1)
      and the same K normals (label=0) from the same split.
    • Concatenates everything and shuffles, giving a balanced batch with
      equal numbers of OOD and ID samples.

    Returns:
        Xq: (N, D) float32
        yq: (N,) int64 with 1 = OOD (anomaly), 0 = ID (normal)
        used: list[str] of families actually used
    """
    if not ood_fams:
        D = cache.X[next(iter(cache.X))].shape[1]
        return np.empty((0, D), np.float32), np.empty((0,), np.int64), []

    X_pos_chunks, X_neg_chunks, used = [], [], []

    for fam in ood_fams:
        # how many anomalies / normals are available on this split?
        n_pos = cache.count(fam, 1, part)
        n_neg = cache.count(fam, 0, part)

        if n_pos <= 0 or n_neg <= 0:
            continue

        hi = min(n_pos, n_neg, per_class_max)
        if hi < per_class_min:
            # not enough examples to hit the lower bound → skip this family
            continue

        k = int(rng.integers(per_class_min, hi + 1))

        Xa = cache.sample(fam, 1, part, k, rng)
        Xn = cache.sample(fam, 0, part, k, rng)

        X_pos_chunks.append(Xa.astype(np.float32, copy=False))
        X_neg_chunks.append(Xn.astype(np.float32, copy=False))
        used.append(fam)

    # Nothing usable → empty outer batch
    if len(X_pos_chunks) == 0 or len(X_neg_chunks) == 0:
        D = cache.X[next(iter(cache.X))].shape[1]
        return np.empty((0, D), np.float32), np.empty((0,), np.int64), []

    # Concatenate per-family chunks
    X_pos = np.concatenate(X_pos_chunks, axis=0).astype(np.float32, copy=False)
    X_neg = np.concatenate(X_neg_chunks, axis=0).astype(np.float32, copy=False)

    # Global balance: equal #pos / #neg
    m = min(len(X_pos), len(X_neg))
    ip = rng.choice(len(X_pos), size=m, replace=False)
    in_ = rng.choice(len(X_neg), size=m, replace=False)

    Xq = np.concatenate([X_pos[ip], X_neg[in_]], axis=0)
    yq = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)], axis=0)

    # Shuffle the batch
    perm = rng.permutation(len(Xq))
    Xq, yq = Xq[perm], yq[perm]

    return Xq, yq, used



In [ ]:


# ------------------------ Metrics / eval & Calibration ------------------------
@torch.no_grad()
def odin_pnorm_numpy(X_np, encoder, clf, odin, bs: int = 8192, steps: int = VAL_PGD_STEPS):
    encoder.eval(); clf.eval(); odin.eval()
    X_np = X_np.astype(np.float32, copy=False)
    outs = []
    for i in range(0, X_np.shape[0], bs):
        xb = torch.from_numpy(X_np[i:i+bs]).to(device).float()
        with torch.enable_grad():
            x_adv = odin_perturb(xb, encoder, clf, odin, steps=steps, tau=ODIN_TAU)
        logits = clf(encoder(x_adv)) / odin.T
        p_norm = torch.sigmoid(logits[:, 0])
        outs.append(p_norm.detach().cpu().numpy())
    return np.concatenate(outs, axis=0)


def _best_f1_with_min_precision(y_true: np.ndarray, scores_anom: np.ndarray, min_prec: float) -> Tuple[float, float]:
    # Scan a dense grid like _best_f1_threshold but require precision >= min_prec
    grid = np.unique(np.concatenate([np.linspace(1e-6, 0.05, 80),
                                     np.linspace(0.05, 0.95, 181),
                                     np.linspace(0.95, 0.999999, 50)]))
    best_t, best_f1 = None, -1.0
    for t in grid:
        yhat = (scores_anom >= t).astype(int)
        prec = precision_score(y_true, yhat, zero_division=0)
        if prec + 1e-12 < float(min_prec):  # fails constraint
            continue
        f1 = f1_score(y_true, yhat, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, float(t)
    # Fallback: if constraint yields nothing, fall back to unconstrained best F1
    if best_t is None:
        return _best_f1_threshold(y_true, scores_anom)
    return best_t, best_f1


def _best_f1_threshold(y_true: np.ndarray, scores_anom: np.ndarray) -> Tuple[float,float]:
    grid = np.unique(np.concatenate([np.linspace(1e-6, 0.05, 80),
                                     np.linspace(0.05, 0.95, 181),
                                     np.linspace(0.95, 0.999999, 50)]))
    best_t, best_f1 = 0.5, -1.0
    for t in grid:
        yhat = (scores_anom >= t).astype(int)
        f1 = f1_score(y_true, yhat, zero_division=0)
        if f1 > best_f1: best_f1, best_t = f1, float(t)
    return best_t, best_f1

def _balanced_block(cache: DataCache, fam: str, rng: np.random.Generator, *, part: str, cap: int):
    m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap)
    if m <= 0: return np.empty((0,0), np.float32), np.empty((0,), np.int64)
    Xa = cache.sample(fam, 1, part, m, rng); Xn = cache.sample(fam, 0, part, m, rng)
    X  = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
    y  = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
    return X, y

# ------------------------ VAL metrics after each episode ------------------------
def _metrics_at_threshold(y_true: np.ndarray, scores_anom: np.ndarray, thr: float):
    """scores_anom: higher = more anomalous"""
    yhat = (scores_anom >= thr).astype(int)
    prec = precision_score(y_true, yhat, zero_division=0)
    rec  = recall_score(y_true, yhat, zero_division=0)
    acc  = accuracy_score(y_true, yhat)
    f1   = f1_score(y_true, yhat, zero_division=0)
    try:
        auc_roc = roc_auc_score(y_true, scores_anom) if (np.unique(y_true).size == 2) else float("nan")
    except Exception:
        auc_roc = float("nan")
    ap = average_precision_score(y_true, scores_anom) if y_true.sum() > 0 else float("nan")
    # PR-AUC (area under PR curve). AP is already the average precision; we show both.
    try:
        P, R, _ = precision_recall_curve(y_true, scores_anom)
        pr_auc = auc(R, P)
    except Exception:
        pr_auc = float("nan")
    return dict(prec=prec, rec=rec, acc=acc, f1=f1, auc_roc=auc_roc, ap=ap, pr_auc=pr_auc)

@torch.no_grad()
def log_val_stats_after_episode(
    cache: DataCache,
    encoder: nn.Module,
    clf: nn.Module,
    odin: ODINParams,
    rng: np.random.Generator,
    *,
    part: str = "val",
    cap_per_class: int = VAL_PER_CLASS_CAP,
    steps: int = VAL_PGD_STEPS,
    tag: str = ""
):
    """
    Computes pooled and per-family metrics on `part` (val/test) using ODIN scores.
    - Picks a single global threshold by maximizing F1 on the *pooled* set.
    - Reports pooled metrics and per-family metrics at that global threshold.
    - Uses up to `cap_per_class` ID and OOD per family (balanced block).
    """
    fams = sorted(cache.files.keys())

    # Build pooled evaluation set
    X_blocks, y_blocks, used_counts = [], [], {}
    for fam in fams:
        m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap_per_class)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng)
        Xn = cache.sample(fam, 0, part, m, rng)
        Xf = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        yf = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
        X_blocks.append(Xf); y_blocks.append(yf)
        used_counts[fam] = (m, m)

    if not X_blocks:
        print("[VAL-METRICS] no data for split:", part)
        return None

    X_all = np.vstack(X_blocks)
    y_all = np.concatenate(y_blocks)

    # Get ODIN "normal prob" then convert to anomaly score
    p_norm = odin_pnorm_numpy(X_all, encoder, clf, odin, steps=steps)
    scores = 1.0 - p_norm  # higher = more anomalous

    # Pick pooled best-F1 threshold and compute pooled metrics
    thr_star, _ = _best_f1_threshold(y_all, scores)
    pooled = _metrics_at_threshold(y_all, scores, thr_star)

    print(f"[VAL-METRICS] {part.upper()} {tag} | thr*={thr_star:.6f} | "
          f"Prec={pooled['prec']:.3f} Rec={pooled['rec']:.3f} Acc={pooled['acc']:.3f} "
          f"F1={pooled['f1']:.3f} AP={pooled['ap']:.3f} PR-AUC={pooled['pr_auc']:.3f} "
          f"AUC-ROC={pooled['auc_roc']:.3f}")

    # Per-family metrics @ the same global threshold
    print("[VAL-METRICS per-family @ global thr*]")
    off = 0
    for fam in fams:
        if fam not in used_counts:
            print(f"  📉 {fam:<12} | (no data)")
            continue
        m_id, m_ood = used_counts[fam]
        n = m_id + m_ood
        yf = y_all[off:off+n]
        sf = scores[off:off+n]
        off += n
        stats = _metrics_at_threshold(yf, sf, thr_star)
        print(f"  📊 {fam:<12} | Prec={stats['prec']:.3f} Rec={stats['rec']:.3f} "
              f"Acc={stats['acc']:.3f} F1={stats['f1']:.3f} "
              f"AP={stats['ap']:.3f} PR-AUC={stats['pr_auc']:.3f} "
              f"AUC-ROC={stats['auc_roc']:.3f}  (used ID={m_id:,} OOD={m_ood:,})")

    # Return in case you want to reuse the threshold elsewhere in your loop
    return float(thr_star), pooled



def test_per_family(cache: DataCache, encoder, clf, odin, thr_star: float, rng: np.random.Generator):
    print("\n===== FINAL TEST (per-family, balanced, disjoint) =====")
    fams = list(cache.files.keys())
    all_scores, all_labels = [], []
    for fam in fams:
        Xf, yf = _balanced_block(cache, fam, rng, part="test", cap=TEST_PER_CLASS_CAP)
        if Xf.size == 0:
            print(f"  🧪 {fam:<12} | not enough samples; skip"); continue
        p_norm = odin_pnorm_numpy(Xf, encoder, clf, odin, steps=VAL_PGD_STEPS)
        scores = 1.0 - p_norm
        yhat = (scores >= thr_star).astype(int)
        prec = precision_score(yf, yhat, zero_division=0)
        rec  = recall_score(yf, yhat, zero_division=0)
        acc  = accuracy_score(yf, yhat)
        f1   = f1_score(yf, yhat, zero_division=0)
        ap   = average_precision_score(yf, scores) if yf.sum() > 0 else float("nan")
        try: auc_roc = roc_auc_score(yf, scores) if (np.unique(yf).size == 2) else float("nan")
        except Exception: auc_roc = float("nan")
        print(f"  🧪 {fam:<12} | Prec={prec:.3f}  Rec={rec:.3f}  Acc={acc:.3f}  F1={f1:.3f}  AP={ap:.3f}  AUC-ROC={auc_roc:.3f}")
        all_scores.append(scores); all_labels.append(yf)

    if all_scores:
        scores = np.concatenate(all_scores); labels = np.concatenate(all_labels)
        yhat = (scores >= thr_star).astype(int)
        prec = precision_score(labels, yhat, zero_division=0)
        rec  = recall_score(labels, yhat, zero_division=0)
        acc  = accuracy_score(labels, yhat)
        f1   = f1_score(labels, yhat, zero_division=0)
        ap   = average_precision_score(labels, scores) if labels.sum() > 0 else float("nan")
        pr_p, pr_r, _ = precision_recall_curve(labels, scores)
        auc_pr = auc(pr_r, pr_p) if len(pr_r) > 1 else float("nan")
        try:
            auc_roc = roc_auc_score(labels, scores) if (np.unique(labels).size == 2) else float("nan")
        except Exception:
            auc_roc = float("nan")
        print(f"\n[OVERALL TEST] thr={thr_star:.6f} | "
              f"Prec={prec:.3f}  Rec={rec:.3f}  Acc={acc:.3f}  F1={f1:.3f}  "
              f"AP={ap:.3f}  AUC-PR={auc_pr:.3f}  AUC-ROC={auc_roc:.3f}")
    else:
        print("\n[OVERALL TEST] No eligible families/samples on test split.")



# ------------------------ Simple VAL/TEST counters (drop-in for validate_robust) ------------------------
def validate_robust(
    cache: DataCache,
    encoder, clf, odin,
    rng: np.random.Generator,print_counts=False,
    *,
    show_test: bool = True,
    cap_val:  Optional[int] = VAL_PER_CLASS_CAP,
    cap_test: Optional[int] = TEST_PER_CLASS_CAP,
):
    """
    Simple counter-only replacement to debug data usage.
    Logs how many ID(0) and OOD(1) samples are available/used per family in VAL/TEST.
    Returns a neutral 3-tuple to keep the caller happy:
        (robust_thr, pooled_thr, per_family_dict)
    where robust_thr = pooled_thr = 0.5 and per_family_dict is an empty dict.
    """

    def _summarize(part: str, cap: Optional[int]):
      if print_counts:
        fams = sorted(cache.files.keys())
        total_id = total_ood = 0
        print(f"[VAL-COUNT] {part.upper():>5}", end=" ")
        # compute totals first (with caps if provided)
        for fam in fams:
            id_cnt  = cache.count(fam, 0, part)
            ood_cnt = cache.count(fam, 1, part)
            if cap is not None:
                id_cnt  = min(id_cnt,  cap)
                ood_cnt = min(ood_cnt, cap)
            total_id  += id_cnt
            total_ood += ood_cnt
        print(f"| used ID={total_id:,}  OOD={total_ood:,}")

        # per-family lines
        for fam in fams:
            id_cnt  = cache.count(fam, 0, part)
            ood_cnt = cache.count(fam, 1, part)
            if cap is not None:
                id_cnt  = min(id_cnt,  cap)
                ood_cnt = min(ood_cnt, cap)
            print(f"  [VAL-COUNT] {fam:<12} | used ID={id_cnt:,}  OOD={ood_cnt:,}")

    # Always show VAL with its cap
    _summarize("val", cap_val)

    # Optionally show TEST too
    if show_test:
        _summarize("test", cap_test)

    # Return dummy thresholds so existing call:
    #   thr_robust, thr_pooled, _ = validate_robust(...)
    # keeps working while you focus on margin effects.
    robust_thr = 0.5
    pooled_thr = 0.5
    per_family = {}
    return robust_thr, pooled_thr, per_family



import numpy as np
import matplotlib.pyplot as plt

def plot_roc_curves(fpr_dict, tpr_dict, auc_dict, title="ROC comparison", save_path="roc.png"):
    """
    fpr_dict: dict of {name: fpr_values}
    tpr_dict: dict of {name: tpr_values}
    auc_dict: dict of {name: auc_value}
    """

    # shrink width (figsize[0]) but keep height similar
    fig, ax = plt.subplots(figsize=(5, 4))   # was (8,6) or larger

    for name in fpr_dict.keys():
        ax.plot(
            fpr_dict[name],
            tpr_dict[name],
            lw=2,
            label=f"{name}  (AUC={auc_dict[name]:.3f})"
        )

    ax.plot([0, 1], [0, 1], "k--", lw=1)

    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel("False positive rate", fontsize=12)
    ax.set_ylabel("True positive rate", fontsize=12)
    ax.set_title(title, fontsize=13)

    # force x-axis ticks at 0.0, 0.4, 0.8, 1.0
    ax.set_xticks(np.arange(0, 1.01, 0.4))

    ax.legend(loc="lower right", fontsize=11, frameon=True)

    fig.tight_layout()
    fig.savefig(save_path, dpi=200, bbox_inches="tight")
    plt.close(fig)
    print(f"[PLOT] ROC figure saved to {save_path}")




def plot_id_ood_separation(
    cache, encoder, clf, odin, rng,
    part="val", families=None, cap_per_family=5000, bins=60,
    episode=None, save_dir="plots", show=False,
):
    import os, numpy as np, matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, precision_recall_curve, auc, average_precision_score

    # helper: balanced block
    def _balanced_block(fam):
        m = min(cache.count(fam,1,part), cache.count(fam,0,part), cap_per_family)
        if m <= 0: return np.empty((0,0),np.float32), np.empty((0,),np.int64)
        Xa = cache.sample(fam,1,part,m,rng)
        Xn = cache.sample(fam,0,part,m,rng)
        X = np.vstack([Xa,Xn]).astype(np.float32, copy=False)
        y = np.concatenate([np.ones(m,np.int64), np.zeros(m,np.int64)])
        return X,y

    fams = families if families else list(cache.files.keys())
    X_pool,y_pool=[],[]
    for fam in fams:
        Xf,yf=_balanced_block(fam)
        if Xf.size>0: X_pool.append(Xf); y_pool.append(yf)
    if not X_pool:
        return {"path":None,"scores_id":[], "scores_ood":[], "roc_auc":float("nan"), "ap":float("nan")}

    Xb = np.vstack(X_pool); yb=np.concatenate(y_pool)
    p_norm = odin_pnorm_numpy(Xb, encoder, clf, odin, steps=1)
    scores = 1.0 - p_norm
    mask=np.isfinite(scores); scores,yb=scores[mask], yb[mask]

    scores_id=scores[yb==0]; scores_ood=scores[yb==1]
    fpr,tpr,_=roc_curve(yb,scores); roc_auc=auc(fpr,tpr)
    pr_p,pr_r,_=precision_recall_curve(yb,scores); ap=average_precision_score(yb,scores)

    fig=plt.figure(figsize=(6,5))
    ax1=plt.subplot(2,2,1); ax2=plt.subplot(2,2,2); ax3=plt.subplot(2,1,2)

    if scores_id.size: ax1.hist(scores_id,bins=bins,alpha=0.6,label="ID",density=True)
    if scores_ood.size: ax1.hist(scores_ood,bins=bins,alpha=0.6,label="OOD",density=True)
    ax1.set_title("Score distribution"); ax1.set_xlabel("Anomaly score"); ax1.set_ylabel("Density")
    ax1.legend()

    ax2.plot(fpr,tpr,lw=2,label=f"AUC={roc_auc:.3f}"); ax2.plot([0,1],[0,1],"k--",lw=1)
    ax2.set_title("ROC"); ax2.set_xlabel("FPR"); ax2.set_ylabel("TPR"); ax2.legend()

    ax3.plot(pr_r,pr_p,lw=2,label=f"AP={ap:.3f}")
    ax3.set_title("Precision–Recall"); ax3.set_xlabel("Recall"); ax3.set_ylabel("Precision"); ax3.legend()

    # force xticks to 0.0,0.4,0.8,1.0 everywhere
    for ax in [ax1,ax2,ax3]:
        ax.set_xlim(0,1); ax.set_xticks([0.0,0.4,0.8,1.0])

    fig.tight_layout()
    path=None
    if save_dir:
        os.makedirs(save_dir,exist_ok=True)
        fname="sep_latest.png" if episode is None else f"sep_ep{int(episode):02d}.png"
        path=os.path.join(save_dir,fname)
        fig.savefig(path,dpi=200,bbox_inches=None,pad_inches=0.05)  # no "tight"
    if show: plt.show()
    plt.close(fig)
    return {"path":path,"scores_id":scores_id,"scores_ood":scores_ood,"roc_auc":float(roc_auc),"ap":float(ap)}

def _part_counts(cache, fam: str, part: str):
    return cache.count(fam, 0, part), cache.count(fam, 1, part)  # (ID_neg, OOD_pos)




In [ ]:
# ===== Per-family thresholds ==================================================
from typing import Dict, Tuple, Optional, List
import numpy as np
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score,
    average_precision_score, roc_auc_score, precision_recall_curve, auc
)

# Reuse your existing helpers if they already exist:
# _best_f1_threshold(y_true, scores)
# _best_f1_with_min_precision(y_true, scores, min_prec)

@torch.no_grad()
def _collect_scores_by_family(
    cache, encoder, clf, odin,
    rng: np.random.Generator,
    *,
    part: str,
    cap_per_class: int,
    steps: int
) -> Tuple[Dict[str, np.ndarray], Dict[str, np.ndarray], np.ndarray, np.ndarray, Dict[str, Tuple[int,int]]]:
    """
    Returns:
      y_by_fam[fam] -> (n_fam,)
      s_by_fam[fam] -> (n_fam,)  (scores; higher == more anomalous)
      y_all, s_all  -> pooled
      used_counts[fam] -> (m_id, m_ood)
    """
    encoder.eval(); clf.eval(); odin.eval()

    fams = sorted(cache.files.keys())
    y_by_fam, s_by_fam, used_counts = {}, {}, {}

    all_y, all_s = [], []

    for fam in fams:
        m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap_per_class)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng)
        Xn = cache.sample(fam, 0, part, m, rng)
        Xf = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        yf = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])

        # ODIN scores
        # p_norm in [0,1]; anomaly score = 1 - p_norm
        xb = torch.from_numpy(Xf).to(next(clf.parameters()).device).float()
        p_norm = odin_score_eval(xb, encoder, clf, odin, steps=steps)  # [B]
        sf = (1.0 - p_norm).cpu().numpy()

        y_by_fam[fam] = yf
        s_by_fam[fam] = sf
        used_counts[fam] = (m, m)
        all_y.append(yf); all_s.append(sf)

    if not all_y:
        return {}, {}, np.empty(0, np.int64), np.empty(0, np.float32), {}

    y_all = np.concatenate(all_y); s_all = np.concatenate(all_s)
    return y_by_fam, s_by_fam, y_all, s_all, used_counts


def pick_thresholds_per_family(
    y_by_fam: Dict[str, np.ndarray],
    s_by_fam: Dict[str, np.ndarray],
    *,
    min_precision: Optional[float] = None,
    id_ceiling_pct: Optional[float] = None
) -> Dict[str, float]:
    """
    Pick a threshold per family. If min_precision is set, use the constrained
    F1 search; otherwise normal best-F1. Optionally clamp by the given percentile
    of ID scores (to avoid exploding thresholds when positives are scarce).
    """
    thr_by_fam: Dict[str, float] = {}
    for fam, y in y_by_fam.items():
        s = s_by_fam[fam]
        if y.size == 0:
            continue
        if min_precision is None:
            t, _ = _best_f1_threshold(y, s)
        else:
            t, _ = _best_f1_with_min_precision(y, s, float(min_precision))

        if id_ceiling_pct is not None:
            s_id = s[y == 0]
            if s_id.size > 0:
                ceiling = np.percentile(s_id, float(id_ceiling_pct))
                t = min(t, float(ceiling))
        thr_by_fam[fam] = float(t)
    return thr_by_fam


def _metrics_at_threshold(y_true: np.ndarray, scores: np.ndarray, thr: float):
    yhat = (scores >= thr).astype(int)
    prec = precision_score(y_true, yhat, zero_division=0)
    rec  = recall_score(y_true, yhat, zero_division=0)
    acc  = accuracy_score(y_true, yhat)
    f1   = f1_score(y_true, yhat, zero_division=0)
    try:
        auc_roc = roc_auc_score(y_true, scores) if (np.unique(y_true).size == 2) else float("nan")
    except Exception:
        auc_roc = float("nan")
    ap = average_precision_score(y_true, scores) if y_true.sum() > 0 else float("nan")
    try:
        P, R, _ = precision_recall_curve(y_true, scores)
        pr_auc = auc(R, P)
    except Exception:
        pr_auc = float("nan")
    return dict(prec=prec, rec=rec, acc=acc, f1=f1, auc_roc=auc_roc, ap=ap, pr_auc=pr_auc)



In [ ]:
# ===================== Per-family metrics collector =====================

import os, json, math, numpy as np
import matplotlib.pyplot as plt

# Reuse your existing helpers if already defined:
# - _balanced_block
# - _metrics_at_threshold
# - odin_pnorm_numpy
# - VAL_PGD_STEPS, VAL_PER_CLASS_CAP
# If any of those names differ in your code, keep the bodies the same and just change the names here.

def compute_val_metrics_per_family(
    cache,
    encoder,
    clf,
    odin,
    rng: np.random.Generator,
    *,
    part: str = "val",
    cap_per_class: int = VAL_PER_CLASS_CAP,
    steps: int = VAL_PGD_STEPS,
):
    """
    Non-intrusive clone of your printed validation logic that RETURNS the numbers.
    Returns:
      thr_star: float
      pooled: dict of pooled metrics
      per_family: dict[fam] -> dict(prec, rec, auc_roc, f1, ap, pr_auc, acc, n_id, n_ood)
    """
    fams = sorted(cache.files.keys())

    # Build pooled evaluation set
    X_blocks, y_blocks, used_counts = [], [], {}
    for fam in fams:
        m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap_per_class)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng)
        Xn = cache.sample(fam, 0, part, m, rng)
        Xf = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        yf = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
        X_blocks.append(Xf); y_blocks.append(yf)
        used_counts[fam] = (m, m)

    if not X_blocks:
        return None, None, {}

    X_all = np.vstack(X_blocks)
    y_all = np.concatenate(y_blocks)

    # ODIN normal prob → anomaly score
    p_norm = odin_pnorm_numpy(X_all, encoder, clf, odin, steps=steps)
    scores = 1.0 - p_norm

    # Same pooled best-F1 threshold as your logger
    thr_star, _ = _best_f1_threshold(y_all, scores)
    pooled = _metrics_at_threshold(y_all, scores, thr_star)

    # Per-family stats @ global thr*
    per_family = {}
    off = 0
    for fam in fams:
        if fam not in used_counts:
            continue
        m_id, m_ood = used_counts[fam]
        n = m_id + m_ood
        yf = y_all[off:off+n]
        sf = scores[off:off+n]
        off += n
        stats = _metrics_at_threshold(yf, sf, thr_star)
        per_family[fam] = dict(
            prec=float(stats["prec"]), rec=float(stats["rec"]), auc_roc=float(stats["auc_roc"]),
            f1=float(stats["f1"]), ap=float(stats["ap"]), pr_auc=float(stats["pr_auc"]),
            acc=float(stats["acc"]), n_id=int(m_id), n_ood=int(m_ood),
        )
    return float(thr_star), pooled, per_family


class PerFamMetricHistory:
    """
    Holds metric curves over episodes. Non-invasive: you create one and call .update(...)
    after each episode. Save to disk optionally.
    """
    def __init__(self, fam_names=None):
        self.history = {}
        if fam_names is not None:
            for f in fam_names:
                self._ensure_family(f)

    def _ensure_family(self, fam):
        if fam not in self.history:
            self.history[fam] = {
                "episode": [],
                "prec": [],
                "rec": [],
                "auc_roc": [],
            }

    def update(self, episode_idx: int, per_family_stats: dict):
        # per_family_stats: {fam: {prec, rec, auc_roc, ...}}
        for fam, st in per_family_stats.items():
            self._ensure_family(fam)
            self.history[fam]["episode"].append(int(episode_idx))
            self.history[fam]["prec"].append(float(st.get("prec", float("nan"))))
            self.history[fam]["rec"].append(float(st.get("rec", float("nan"))))
            self.history[fam]["auc_roc"].append(float(st.get("auc_roc", float("nan"))))

    def to_json(self):
        return json.dumps(self.history, indent=2)

    def save_json(self, path: str):
        os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
        with open(path, "w", encoding="utf-8") as f:
            json.dump(self.history, f, indent=2)

    @classmethod
    def load_json(cls, path: str):
        obj = cls()
        with open(path, "r", encoding="utf-8") as f:
            obj.history = json.load(f)
        return obj


def _slugify(text: str) -> str:
    # File-name friendly
    return "".join(ch if ch.isalnum() else "_" for ch in text).strip("_")[:140]


def plot_per_family_metric_curves(history: PerFamMetricHistory, out_dir: str = "plots_perfam_curves"):
    """
    Produces one figure per family with 3 curves: Precision, Recall, AUC-ROC vs episodes.
    """
    os.makedirs(out_dir, exist_ok=True)
    for fam, recs in history.history.items():
        if not recs["episode"]:
            continue
        xs = recs["episode"]
        ys_prec = recs["prec"]
        ys_rec  = recs["rec"]
        ys_auc  = recs["auc_roc"]

        plt.figure(figsize=(6, 4))
        plt.plot(xs, ys_prec, label="Precision")
        plt.plot(xs, ys_rec,  label="Recall")
        plt.plot(xs, ys_auc,  label="AUC-ROC")
        plt.ylim(-0.05, 1.05)
        plt.xlabel("Episode")
        plt.ylabel("Score")
        plt.title(f"{fam} — Precision / Recall / AUC-ROC vs Episode")
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.legend()
        fname = os.path.join(out_dir, f"{_slugify(fam)}.png")
        plt.tight_layout()
        plt.savefig(fname, dpi=150)
        plt.close()


In [ ]:
from collections import defaultdict
from dataclasses import dataclass
import numpy as np
import torch

# ---------- A) Fixed global threshold at a target FPR ----------

def _extract_labels_scores_from_valdump(val_dump):
    """
    Make this tolerant to your current validate_robust() payload.
    It tries the common shapes you've been using.
    Returns y (0=OOD, 1=ID), scores (higher => more normal).
    """
    if isinstance(val_dump, dict):
        if "y_true" in val_dump and "scores" in val_dump:
            y = np.asarray(val_dump["y_true"])
            s = np.asarray(val_dump["scores"])
            return y, s
        if "ys" in val_dump and "scores" in val_dump:
            y = np.asarray(val_dump["ys"])
            s = np.asarray(val_dump["scores"])
            return y, s
        if "id_scores" in val_dump and "ood_scores" in val_dump:
            id_s  = np.asarray(val_dump["id_scores"])
            ood_s = np.asarray(val_dump["ood_scores"])
            y = np.concatenate([np.ones_like(id_s), np.zeros_like(ood_s)])
            s = np.concatenate([id_s, ood_s])
            return y, s
    raise ValueError("Unrecognized val_dump structure; expose y_true/scores or id_scores/ood_scores.")



@dataclass
class FamilyCoverageScheduler:
    all_fams: list
    heldout_fams: set

    def __post_init__(self):
        self.heldout_fams = set(self.heldout_fams)
        self.outer_counts = defaultdict(int)  # times used as OOD in outer meta loss
        self.inner_counts = defaultdict(int)  # times included in inner clean-ID push-up

    def pick_for_inner(self, allowed_fams, inner_min_fams, inner_max_fams, rng):
        """Prioritize families with lowest inner_counts."""
        cand = [f for f in allowed_fams if f not in self.heldout_fams]
        if not cand:
            return []
        cand.sort(key=lambda f: (self.inner_counts[f], rng.random()))
        k = max(inner_min_fams, min(inner_max_fams, len(cand)))
        chosen = cand[:k]
        for f in chosen:
            self.inner_counts[f] += 1
        return chosen

    def pick_for_outer(self, allowed_fams, k, rng, priority=None):
        """
        Prioritize (1) explicit priority list, then (2) lowest outer_counts.
        """
        allowed = [f for f in allowed_fams if f not in self.heldout_fams]
        if not allowed or k <= 0:
            return []
        priority = priority or []
        prio_set = set(priority)

        def keyfunc(f):
            # priority families get rank 0, others 1; then by count, then by random tiebreaker
            return (0 if f in prio_set else 1, self.outer_counts[f], rng.random())

        allowed.sort(key=keyfunc)
        chosen = allowed[:k]
        for f in chosen:
            self.outer_counts[f] += 1
        return chosen

# ---------- C) Small helper to print metrics at a fixed thr using your existing validation ----------
def print_val_metrics_at_fixed_thr(val_dump, thr, tag="(FIXED-THR)"):
    y, s = _extract_labels_scores_from_valdump(val_dump)
    y = (y.astype(int))
    pred = (s >= thr).astype(int)  # 1=ID, 0=OOD

    # basic metrics (global)
    tp = int(((pred == 1) & (y == 1)).sum())
    fp = int(((pred == 1) & (y == 0)).sum())
    tn = int(((pred == 0) & (y == 0)).sum())
    fn = int(((pred == 0) & (y == 1)).sum())

    prec = tp / max(1, (tp + fp))
    rec  = tp / max(1, (tp + fn))
    acc  = (tp + tn) / max(1, len(y))
    f1   = (2 * prec * rec) / max(1e-8, (prec + rec))

    # FPR for reference (based on OOD)
    ood_ct = max(1, (y == 0).sum())
    fpr = fp / ood_ct

    print(f"[VAL-METRICS] {tag} | thr={thr:.6f} | Prec={prec:.3f} Rec={rec:.3f} Acc={acc:.3f} F1={f1:.3f} FPR={fpr:.3f}")


In [ ]:
import contextlib

# ---------- (A) Robust (y, s) extractor from your existing val_dump ----------
def extract_y_scores_from_valdump(val_dump):
    """
    Returns y (0=OOD, 1=ID) and anomaly scores s (higher = more anomalous).
    Works with either {'y_true','scores'} or {'id_scores','ood_scores'}.
    """
    if isinstance(val_dump, dict):
        if ('y_true' in val_dump) and ('scores' in val_dump):
            y = np.asarray(val_dump['y_true']).astype(int)
            s = np.asarray(val_dump['scores']).astype(float)
            return y, s
        if ('id_scores' in val_dump) and ('ood_scores' in val_dump):
            id_s  = np.asarray(val_dump['id_scores']).astype(float)
            ood_s = np.asarray(val_dump['ood_scores']).astype(float)
            y = np.concatenate([np.ones_like(id_s, int), np.zeros_like(ood_s, int)])
            s = np.concatenate([id_s, ood_s]).astype(float)
            return y, s
    raise RuntimeError("Unrecognized val_dump structure; needs ('y_true','scores') or ('id_scores','ood_scores').")


# ---------- (B) Post-hoc calibration metrics and utilities (pure eval) ----------
from sklearn.calibration import calibration_curve
from sklearn.metrics import log_loss

def sigmoid_clip(z, eps=1e-6):
    p = 1. / (1. + np.exp(-z))
    return np.clip(p, eps, 1.0 - eps)

def logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1.0 - eps)
    return np.log(p) - np.log(1. - p)

def compute_calibration_metrics(y, s_anom, n_bins=15):
    """
    Treat ID probability as p_id = 1 - s_anom (your ODIN score is anomaly-like).
    Returns dict with ECE, NLL, Brier, curves for plotting.
    """
    p_id = np.clip(1.0 - s_anom, 1e-6, 1.0 - 1e-6)
    # NLL on binary labels (ID==1)
    nll   = float(log_loss(y, p_id, labels=[0,1]))
    # Brier (binary)
    brier = float(np.mean((p_id - y.astype(float))**2))
    # ECE
    prob_true, prob_pred = calibration_curve(y, p_id, n_bins=n_bins, strategy="uniform")
    # expected calibration error (uniform bins)
    # note: sklearn doesn't return bin weights; approximate by averaging abs gap
    ece = float(np.mean(np.abs(prob_pred - prob_true)))
    return dict(nll=nll, brier=brier, ece=ece,
                cal_pred=prob_pred, cal_true=prob_true)

@torch.no_grad()
def posthoc_fit_temperature_on_id(y, s_anom, max_iter=50, lr=0.1):
    """
    Simple post-hoc T fit on *ID only* (y==1). Work in logit space of p_id = 1 - s_anom.
    We find T that minimizes ID-only NLL of sigmoid(z/T).
    """
    p_id = np.clip(1.0 - s_anom, 1e-6, 1.0 - 1e-6)
    z    = torch.tensor(logit_clip(p_id), dtype=torch.float32)
    y_id = torch.tensor((y == 1).astype(np.float32))
    mask = (y_id == 1.0)
    if mask.sum() == 0:
        return 1.0  # nothing to fit

    T = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)
    opt = torch.optim.LBFGS([T], lr=lr, max_iter=max_iter, line_search_fn="strong_wolfe")

    bce = torch.nn.BCEWithLogitsLoss()

    def closure():
        opt.zero_grad(set_to_none=True)
        zT   = z[mask] / T.clamp(min=1e-4, max=100.0)
        loss = bce(zT, y_id[mask])
        loss.backward()
        return loss

    try:
        opt.step(closure)
        T_star = float(T.detach().clamp(1e-4, 100.0).item())
    except Exception:
        T_star = 1.0
    return T_star

@contextlib.contextmanager
def _temporary_set_eps(odin, new_eps: float):
    """Temporarily set ODIN epsilon (in-place) for evaluation only."""
    old = float(odin.eps.detach().item()) if hasattr(odin.eps, "detach") else float(odin.eps)
    try:
        if hasattr(odin, "eps"):
            odin.eps.data.fill_(float(new_eps))
        yield
    finally:
        if hasattr(odin, "eps"):
            odin.eps.data.fill_(float(old))


In [ ]:
#

In [ ]:
# ===== Global VAL helpers: robust extractor + fallback builder + safe printer =====
import numpy as np
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score,
    precision_recall_curve, average_precision_score, roc_auc_score, auc
)

def _extract_labels_scores_from_valdump(val_dump):
    """
    Return y (1=ID/normal, 0=OOD/anomaly) and scores (float) from multiple possible val_dump layouts.
    """
    if val_dump is None:
        raise ValueError("val_dump is None")

    if isinstance(val_dump, dict):
        # Canonical
        if ("y_true" in val_dump) and ("scores" in val_dump):
            y = np.asarray(val_dump["y_true"]).astype(np.int32)
            s = np.asarray(val_dump["scores"]).astype(np.float32)
            return y, s

        # Pooled (id_scores/ood_scores)
        for idk, oodk in [("id_scores", "ood_scores"), ("id", "ood")]:
            if idk in val_dump and oodk in val_dump:
                id_s  = np.asarray(val_dump[idk]).astype(np.float32)
                ood_s = np.asarray(val_dump[oodk]).astype(np.float32)
                y = np.concatenate([np.ones(id_s.shape[0], np.int32),
                                    np.zeros(ood_s.shape[0], np.int32)])
                s = np.concatenate([id_s, ood_s]).astype(np.float32)
                return y, s

        # Nested under "pooled"
        if "pooled" in val_dump and isinstance(val_dump["pooled"], dict):
            pooled = val_dump["pooled"]
            if ("y_true" in pooled) and ("scores" in pooled):
                y = np.asarray(pooled["y_true"]).astype(np.int32)
                s = np.asarray(pooled["scores"]).astype(np.float32)
                return y, s

    raise KeyError("Unrecognized val_dump structure; expose y_true/scores or id_scores/ood_scores.")


def _metrics_at_threshold(y_true, scores, thr):
    yhat = (scores >= float(thr)).astype(np.int32)  # 1=ID (normal)
    prec = precision_score(y_true, yhat, zero_division=0)
    rec  = recall_score(y_true, yhat, zero_division=0)
    acc  = accuracy_score(y_true, yhat)
    f1   = f1_score(y_true, yhat, zero_division=0)
    try:
        auc_roc = roc_auc_score(y_true, scores) if np.unique(y_true).size == 2 else float("nan")
    except Exception:
        auc_roc = float("nan")
    try:
        P, R, _ = precision_recall_curve(y_true, scores)
        pr_auc = auc(R, P) if len(R) > 1 else float("nan")
        ap     = average_precision_score(y_true, scores) if y_true.sum() > 0 else float("nan")
    except Exception:
        pr_auc, ap = float("nan"), float("nan")
    # NOTE: keys match your old printer (prec/rec/acc/f1)
    return dict(prec=prec, rec=rec, acc=acc, f1=f1, pr_auc=pr_auc, ap=ap, auc_roc=auc_roc)



def _build_global_val_dump_fallback(cache, encoder, clf, odin, rng, *, part="val", cap_per_family=4000, steps=1):
    """
    Build a deterministic global pool if validate_robust failed or returns an unknown layout.
    Uses the SAME rng each episode so the pool is identical across episodes.
    """
    fams = sorted(cache.files.keys())
    ys, ss = [], []
    for fam in fams:
        m_id  = min(cache.count(fam, 1, part), cap_per_family)
        m_ood = min(cache.count(fam, 0, part), cap_per_family)
        m = min(m_id, m_ood)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng)  # ID/normal -> label 1
        Xo = cache.sample(fam, 0, part, m, rng)  # OOD/anomaly -> label 0
        X  = np.vstack([Xa, Xo]).astype(np.float32, copy=False)
        y  = np.concatenate([np.ones(m, np.int32), np.zeros(m, np.int32)])
        xb = torch.from_numpy(X).to(next(clf.parameters()).device)
        p_norm = odin_score_eval(xb, encoder, clf, odin, steps=steps)  # probability of normal
        s = (1.0 - p_norm).cpu().numpy().astype(np.float32)           # anomaly score
        ys.append(y); ss.append(s)
    if not ys:
        raise RuntimeError("Fallback global val pool is empty.")
    y_all = np.concatenate(ys); s_all = np.concatenate(ss)
    return {"y_true": y_all, "scores": s_all}


def print_val_metrics_at_fixed_thr_safe(val_dump, thr, tag: str):
    """
    Always prints the single global line (normal vs anomaly) even if val_dump varies in shape.
    """
    y, s = _extract_labels_scores_from_valdump(val_dump)
    m = _metrics_at_threshold(y, s, thr)
    print(f"[VAL-METRICS] {tag} | thr*={float(thr):.6f} | "
          f"Prec={m['prec']:.3f} Rec={m['rec']:.3f} Acc={m['acc']:.3f} "
          f"F1={m['f1']:.3f} AP={m['ap']:.3f} PR-AUC={m['pr_auc']:.3f} AUC-ROC={m['auc_roc']:.3f}")
    return y, s, m


In [ ]:
def _thr_at_fixed_fpr(y: np.ndarray, s: np.ndarray, target_fpr: float) -> float:
    """
    Threshold to achieve a target FPR on the pooled (ID vs anomaly) task.
    y: 1 = anomaly, 0 = normal
    s: anomaly score (higher == more anomalous)
    FPR = P(predict=anomaly | y=0). To get FPR=f, set thr at the (1-f) quantile of scores on y==0.
    """
    neg = s[y == 0]
    if neg.size == 0:
        return float(np.inf)  # degenerate; nothing normal -> any finite thr yields FPR=0
    q = max(0.0, min(1.0, 1.0 - float(target_fpr)))
    # use 'higher' style behavior for stability across ties
    qs = np.quantile(neg, q)
    return float(qs)


In [ ]:
@torch.no_grad()
def log_val_stats_after_episode(
    cache: DataCache,
    encoder: nn.Module,
    clf: nn.Module,
    odin: ODINParams,
    rng: np.random.Generator,
    *,
    part: str = "val",
    cap_per_class: int = VAL_PER_CLASS_CAP,
    steps: int = VAL_PGD_STEPS,
    tag: str = ""
):
    """
    Rebuilds a fresh validation pool (balanced per-family) using `rng` on every call.
    Computes a single global threshold by max-F1 and prints pooled metrics.
    Returns (thr_star, pooled_metrics_dict). Pooled metrics are *global normal vs anomaly*.
    """
    fams = sorted(cache.files.keys())

    # Build pooled evaluation set (fresh sample every call)
    X_blocks, y_blocks, used_counts = [], [], {}
    for fam in fams:
        m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap_per_class)
        if m <= 0:
            continue
        Xa = cache.sample(fam, 1, part, m, rng)
        Xn = cache.sample(fam, 0, part, m, rng)
        Xf = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        yf = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
        X_blocks.append(Xf); y_blocks.append(yf)
        used_counts[fam] = (m, m)

    if not X_blocks:
        print("[VAL-METRICS] no data for split:", part)
        return None

    X_all = np.vstack(X_blocks)
    y_all = np.concatenate(y_blocks)

    # ODIN "normal prob" -> anomaly score = 1 - p_norm
    p_norm = odin_pnorm_numpy(X_all, encoder, clf, odin, steps=steps)
    scores = 1.0 - p_norm  # higher = more anomalous

    # Pick global best-F1 threshold and compute pooled metrics
    thr_star, _ = _best_f1_threshold(y_all, scores)
    pooled = _metrics_at_threshold(y_all, scores, thr_star)

    print(f"[VAL-METRICS] {part.upper()} {tag} | thr*={thr_star:.6f} | "
          f"Prec={pooled['prec']:.3f} Rec={pooled['rec']:.3f} Acc={pooled['acc']:.3f} "
          f"F1={pooled['f1']:.3f} AP={pooled['ap']:.3f} PR-AUC={pooled['pr_auc']:.3f} "
          f"AUC-ROC={pooled['auc_roc']:.3f}")

    return float(thr_star), pooled


In [ ]:
@torch.no_grad()
def test_with_thresholds(
    cache,
    encoder: nn.Module,
    clf: nn.Module,
    odin: ODINParams,
    thresholds,  # float for global OR Dict[str,float] for per-family
    rng: np.random.Generator,
    *,
    part: str = "test",
    cap_per_class: int = TEST_PER_CLASS_CAP,
    steps: int = VAL_PGD_STEPS
):
    """
    Unchanged API; if `thresholds` is a float it is the chosen global thr (from validation).
    """
    print(f"\n===== FINAL TEST ({'per-family' if isinstance(thresholds, dict) else 'global'} thresholds) =====")
    fams = sorted(cache.files.keys())
    all_scores, all_labels = [], []

    for fam in fams:
        Xf, yf = _balanced_block(cache, fam, rng, part=part, cap=cap_per_class)
        if Xf.size == 0:
            print(f"  🧪 {fam:<20} | not enough samples; skip")
            continue

        xb = torch.from_numpy(Xf.astype(np.float32, copy=False)).to(next(clf.parameters()).device)
        p_norm = odin_score_eval(xb, encoder, clf, odin, steps=steps)
        scores = (1.0 - p_norm).cpu().numpy()

        thr = thresholds.get(fam, 0.5) if isinstance(thresholds, dict) else float(thresholds)
        yhat = (scores >= thr).astype(int)

        prec = precision_score(yf, yhat, zero_division=0)
        rec  = recall_score(yf, yhat, zero_division=0)
        acc  = accuracy_score(yf, yhat)
        f1   = f1_score(yf, yhat, zero_division=0)
        ap   = average_precision_score(yf, scores) if yf.sum() > 0 else float("nan")
        try: auc_roc = roc_auc_score(yf, scores) if (np.unique(yf).size == 2) else float("nan")
        except Exception: auc_roc = float("nan")

        print(f"  🧪 {fam:<20} | Prec={prec:.3f}  Rec={rec:.3f}  Acc={acc:.3f}  "
              f"F1={f1:.3f}  AP={ap:.3f}  AUC-ROC={auc_roc:.3f}")

        all_scores.append(scores); all_labels.append(yf)

    if all_scores:
        scores = np.concatenate(all_scores); labels = np.concatenate(all_labels)
        if isinstance(thresholds, dict):
            # apply each family's threshold to its slice
            yhat_list, i = [], 0
            for fam in fams:
                Xf, yf = _balanced_block(cache, fam, rng, part=part, cap=cap_per_class)
                if Xf.size == 0:
                    continue
                n = yf.shape[0]
                fam_scores = scores[i:i+n]
                thr = thresholds.get(fam, 0.5)
                yhat_list.append((fam_scores >= thr).astype(int))
                i += n
            yhat = np.concatenate(yhat_list)
        else:
            yhat = (scores >= float(thresholds)).astype(int)

        prec = precision_score(labels, yhat, zero_division=0)
        rec  = recall_score(labels, yhat, zero_division=0)
        acc  = accuracy_score(labels, yhat)
        f1   = f1_score(labels, yhat, zero_division=0)
        ap   = average_precision_score(labels, scores) if labels.sum() > 0 else float("nan")
        pr_p, pr_r, _ = precision_recall_curve(labels, scores)
        auc_pr = auc(pr_r, pr_p) if len(pr_r) > 1 else float("nan")
        try: auc_roc = roc_auc_score(labels, scores) if (np.unique(labels).size == 2) else float("nan")
        except Exception: auc_roc = float("nan")

        thr_str = "<per-family>" if isinstance(thresholds, dict) else f"{float(thresholds):.6f}"
        print(f"\n[OVERALL TEST] thr={thr_str} | "
              f"Prec={prec:.3f}  Rec={rec:.3f}  Acc={acc:.3f}  F1={f1:.3f}  "
              f"AP={ap:.3f}  AUC-PR={auc_pr:.3f}  AUC-ROC={auc_roc:.3f}")
    else:
        print("\n[OVERALL TEST] No eligible families/samples on test split.")


In [ ]:
# ---------- Balanced VAL pool from held-out families only ----------
@torch.no_grad()
def build_balanced_val_dump_heldout(
    cache,
    heldout_fams: list[str],
    rng: np.random.Generator,
    *,
    k_per_family: int = 2000,
    part: str = "val",
    steps: int = 1,
    encoder: nn.Module,
    clf: nn.Module,
    odin,                      # ODINParams or similar
    device: torch.device = None,
) -> dict:
    """
    Build a *balanced* validation set strictly from HELD-OUT families:
      - For each held-out family: sample K anomalies and K normals (cap by availability).
      - Compute anomaly scores (HIGHER = more anomalous): s = 1 - p_norm.
    Returns a canonical dict the rest of the code understands:
      {'y_true': np.ndarray[B], 'scores': np.ndarray[B], 'families': list[str]}
    """
    encoder.eval(); clf.eval()
    dev = device or next(clf.parameters()).device

    Ys, Ss, fam_tags = [], [], []
    for fam in sorted(heldout_fams):
        # cap by availability
        k_pos = min(cache.count(fam, 1, part), k_per_family)
        k_neg = min(cache.count(fam, 0, part), k_per_family)
        m = min(k_pos, k_neg)
        if m <= 0:
            continue

        Xa = cache.sample(fam, 1, part, m, rng)
        Xn = cache.sample(fam, 0, part, m, rng)
        X  = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
        y  = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])

        xb = torch.from_numpy(X).to(dev)
        p_norm = odin_score_eval(xb, encoder, clf, odin, steps=steps)  # in [0,1]
        s = (1.0 - p_norm).cpu().numpy()  # higher => more anomalous

        Ys.append(y); Ss.append(s)
        fam_tags.extend([fam] * (2*m))

    if not Ys:
        raise RuntimeError("Held-out BAL VAL: no data available (check splits and heldout families).")

    y_all = np.concatenate(Ys)
    s_all = np.concatenate(Ss)
    return {"y_true": y_all, "scores": s_all, "families": fam_tags}


# ---------- Threshold selection: best-F1 with clipping ----------
def best_f1_threshold_clipped(
    y: np.ndarray,
    s: np.ndarray,
    *,
    qmin: float = 0.005,
    qmax: float = 0.995,
    prefer: str = "high"  # tie-break: choose higher threshold
) -> tuple[float, float]:
    """
    Search F1 over thresholds clipped to [qmin, qmax] of the score range
    to avoid degenerate all-positive/all-negative endpoints.
    Returns (thr*, F1_at_thr*).
    """
    if y.size == 0:
        return 0.5, float("nan")
    # clip search range
    lo = float(np.quantile(s, qmin))
    hi = float(np.quantile(s, qmax))
    # candidate thresholds = unique scores within [lo,hi]
    cand = s[(s >= lo) & (s <= hi)]
    if cand.size == 0:
        cand = s.copy()
    thr_list = np.unique(np.sort(cand))
    if thr_list.size == 0:
        return 0.5, float("nan")

    best_f1, best_thr = -1.0, float(thr_list[0])
    for t in thr_list:
        yhat = (s >= t).astype(int)
        f1 = f1_score(y, yhat, zero_division=0)
        if (f1 > best_f1) or (np.isclose(f1, best_f1) and prefer == "high" and t > best_thr):
            best_f1, best_thr = f1, float(t)
    return best_thr, best_f1


# ---------- Threshold selection: fixed FPR on ID-only ----------
def fixed_fpr_threshold_id(
    y: np.ndarray,
    s: np.ndarray,
    *,
    target_fpr: float = 0.01
) -> float:
    """
    Pick thr so that FPR on ID (y==0) ≈ target_fpr.
    With score polarity "higher => more anomalous", we take thr as the (1 - target_fpr)
    quantile of ID scores.
    """
    s_id = s[y == 0]
    if s_id.size == 0:
        # fallback to global median if we somehow have no ID in the pool
        return float(np.median(s))
    q = max(0.0, min(1.0, 1.0 - float(target_fpr)))
    return float(np.quantile(s_id, q))


# ---------- One-line safe metrics print at a fixed thr ----------
def print_global_val_at_thr(tag: str, y: np.ndarray, s: np.ndarray, thr: float):
    yhat = (s >= thr).astype(int)
    prec = precision_score(y, yhat, zero_division=0)
    rec  = recall_score(y, yhat, zero_division=0)
    acc  = accuracy_score(y, yhat)
    f1   = f1_score(y, yhat, zero_division=0)
    ap   = average_precision_score(y, s) if y.sum() > 0 else float("nan")
    try:
        auc_roc = roc_auc_score(y, s) if np.unique(y).size == 2 else float("nan")
        P, R, T = precision_recall_curve(y, s)
        pr_auc = auc(R, P) if len(R) > 1 else float("nan")
    except Exception:
        auc_roc, pr_auc = float("nan"), float("nan")
    print(f"[VAL-METRICS] {tag} | thr*={thr:.6f} | "
          f"Prec={prec:.3f} Rec={rec:.3f} Acc={acc:.3f} F1={f1:.3f} "
          f"AP={ap:.3f} PR-AUC={pr_auc:.3f} AUC-ROC={auc_roc:.3f}")
    return dict(prec=prec, rec=rec, acc=acc, f1=f1, ap=ap, pr_auc=pr_auc, auc_roc=auc_roc)


In [ ]:
# ========================== CALIBRATION & VALIDATION ==========================

from typing import Dict, Tuple, Optional
import numpy as np
import torch
from torch import nn
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score,
                             average_precision_score, roc_auc_score,
                             precision_recall_curve, auc)

# --- tiny utilities -----------------------------------------------------------

def _balanced_block(cache, fam: str, rng: np.random.Generator, *, part: str, cap: int):
    """Return a balanced block X (2*m, D) and y (2*m,) for a single family."""
    m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap)
    if m <= 0:
        return np.empty((0, cache.X[fam].shape[1]), np.float32), np.empty((0,), np.int64)
    Xa = cache.sample(fam, 1, part, m, rng)  # OOD
    Xn = cache.sample(fam, 0, part, m, rng)  # ID
    X  = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
    y  = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
    return X, y

@torch.no_grad()
def _score_block_anomaly(X: np.ndarray, encoder: nn.Module, clf: nn.Module, odin, *, steps: int = 1) -> np.ndarray:
    """Return anomaly scores s = 1 - p_norm in [0,1]."""
    device = next(clf.parameters()).device
    xb = torch.from_numpy(X).to(device).float()
    p_norm = odin_score_eval(xb, encoder, clf, odin, steps=steps)  # [B] in [0,1]
    return (1.0 - p_norm).cpu().numpy()

def _metrics_at_threshold(y: np.ndarray, s: np.ndarray, thr: float) -> Dict[str, float]:
    yhat = (s >= thr).astype(np.int64)
    prec = precision_score(y, yhat, zero_division=0)
    rec  = recall_score(y, yhat, zero_division=0)
    acc  = accuracy_score(y, yhat)
    f1   = f1_score(y, yhat, zero_division=0)
    try:
        auc_roc = roc_auc_score(y, s) if (np.unique(y).size == 2) else float("nan")
    except Exception:
        auc_roc = float("nan")
    ap = average_precision_score(y, s) if y.sum() > 0 else float("nan")
    try:
        P, R, _ = precision_recall_curve(y, s); pr_auc = auc(R, P)
    except Exception:
        pr_auc = float("nan")
    return dict(prec=prec, rec=rec, acc=acc, f1=f1, ap=ap, pr_auc=pr_auc, auc_roc=auc_roc)

def _best_f1_threshold(y: np.ndarray, s: np.ndarray) -> Tuple[float, float]:
    """Return (thr*, bestF1) for anomaly scores s (higher=worse)."""
    P, R, T = precision_recall_curve(y, s)
    f1 = (2 * P * R) / np.clip(P + R, 1e-9, None)
    k  = int(np.nanargmax(f1))
    thr_star = 0.5 if (T.size == 0) else float(T[max(0, k-1)])  # align PR thresholds to labels
    return thr_star, float(np.nanmax(f1))

def _thr_at_fixed_fpr(y: np.ndarray, s: np.ndarray, fpr: float) -> float:
    """
    Choose thr such that FPR=Pr[s>=thr | y=0] ~= target fpr.
    """
    s_id = s[y == 0]
    if s_id.size == 0:
        return 0.5
    q = np.clip(100.0 * (1.0 - float(fpr)), 0.0, 100.0)
    return float(np.percentile(s_id, q))  # anomaly score percentile on ID

# --- build validation from held-out families ----------------------------------

@torch.no_grad()
def build_val_from_heldout(
    cache,
    heldout_fams: list[str],
    encoder: nn.Module,
    clf: nn.Module,
    odin,
    rng: np.random.Generator,
    *,
    part: str = "val",
    k_per_family: int = 2000,
    steps: int = 1
) -> Tuple[Dict[str, np.ndarray], Dict[str, np.ndarray], np.ndarray, np.ndarray]:
    """
    For *each held-out family*, take K OOD + K ID from split `part`, score them,
    and return {fam: y}, {fam: s}, and pooled (y_all, s_all).
    """
    y_by_fam, s_by_fam = {}, {}
    all_y, all_s = [], []
    for fam in sorted(heldout_fams):
        Xf, yf = _balanced_block(cache, fam, rng, part=part, cap=k_per_family)
        if Xf.size == 0:
            continue
        sf = _score_block_anomaly(Xf, encoder, clf, odin, steps=steps)
        # sanity: sf must be anomaly scores
        assert np.isfinite(sf).all(), "NaNs in anomaly scores"
        y_by_fam[fam] = yf
        s_by_fam[fam] = sf
        all_y.append(yf); all_s.append(sf)

    if not all_y:
        return {}, {}, np.empty((0,), np.int64), np.empty((0,), np.float32)

    y_all = np.concatenate(all_y); s_all = np.concatenate(all_s)
    return y_by_fam, s_by_fam, y_all, s_all

# --- per-episode validation printer -------------------------------------------

def print_global_and_perfam(
    y_by_fam: dict[str, np.ndarray],
    s_by_fam: dict[str, np.ndarray],
    *,
    choose_thr_by: str = "bestf1",   # "bestf1" or "fpr"
    target_fpr: float = 0.01,
    tag: str = "VAL"
) -> tuple[float, float]:
    """
    Prints a pooled 'GLOBAL' line and a per-family table (best-F1 per family).
    Returns (thr_global, f1_global) for the pooled set.
    """
    fams = sorted(y_by_fam.keys())
    y_all = np.concatenate([y_by_fam[f] for f in fams]) if fams else np.empty(0, np.int64)
    s_all = np.concatenate([s_by_fam[f] for f in fams]) if fams else np.empty(0, np.float32)

    if y_all.size == 0:
        print(f"[VAL-METRICS] {tag} (empty)")
        return 0.5, float("nan")

    if choose_thr_by == "fpr":
        thr_star = _thr_at_fixed_fpr(y_all, s_all, target_fpr)
        f1_star  = _metrics_at_threshold(y_all, s_all, thr_star)["f1"]
        tag_thr  = f"TPR@FPR({target_fpr:.1%})"
    else:
        thr_star, f1_star = _best_f1_threshold(y_all, s_all)
        tag_thr = "best-F1"

    m = _metrics_at_threshold(y_all, s_all, thr_star)
    print(f"[VAL-METRICS] {tag} GLOBAL | thr*={thr_star:.6f} ({tag_thr}) | "
          f"Prec={m['prec']:.3f} Rec={m['rec']:.3f} Acc={m['acc']:.3f} "
          f"F1={m['f1']:.3f} AP={m['ap']:.3f} PR-AUC={m['pr_auc']:.3f} AUC-ROC={m['auc_roc']:.3f}")

    print(f"[VAL-METRICS] {tag} PER-FAMILY @ best-F1 (class-specific thresholds)")
    for fam in fams:
        t_f, _ = _best_f1_threshold(y_by_fam[fam], s_by_fam[fam])
        mm = _metrics_at_threshold(y_by_fam[fam], s_by_fam[fam], t_f)
        print(f"  🧪 {fam:<22} | thr={t_f:.6f} | Prec={mm['prec']:.3f} Rec={mm['rec']:.3f} "
              f"Acc={mm['acc']:.3f} F1={mm['f1']:.3f} AP={mm['ap']:.3f} AUC-ROC={mm['auc_roc']:.3f}")

    return float(thr_star), float(f1_star)


In [ ]:
@torch.no_grad()
def final_test_balanced(
    cache,
    encoder: nn.Module,
    clf: nn.Module,
    odin,
    thr_global: float,
    rng: np.random.Generator,
    *,
    part: str = "test",
    cap_per_class: int = 25_000,
    steps: int = 1
):
    print("\n===== FINAL TEST (per-family, balanced, disjoint) =====")
    fams = sorted(cache.files.keys())
    all_scores, all_labels = [], []

    for fam in fams:
        Xf, yf = _balanced_block(cache, fam, rng, part=part, cap=cap_per_class)
        if Xf.size == 0:
            print(f"  🧪 {fam:<22} | not enough samples; skip")
            continue
        scores = _score_block_anomaly(Xf, encoder, clf, odin, steps=steps)
        yhat   = (scores >= thr_global).astype(int)

        prec = precision_score(yf, yhat, zero_division=0)
        rec  = recall_score(yf, yhat, zero_division=0)
        acc  = accuracy_score(yf, yhat)
        f1   = f1_score(yf, yhat, zero_division=0)
        ap   = average_precision_score(yf, scores) if yf.sum() > 0 else float("nan")
        try: auc_roc = roc_auc_score(yf, scores) if (np.unique(yf).size == 2) else float("nan")
        except Exception: auc_roc = float("nan")

        print(f"  🧪 {fam:<22} | Prec={prec:.3f} Rec={rec:.3f} Acc={acc:.3f} "
              f"F1={f1:.3f} AP={ap:.3f} AUC-ROC={auc_roc:.3f}")

        all_scores.append(scores); all_labels.append(yf)

    if all_scores:
        scores = np.concatenate(all_scores); labels = np.concatenate(all_labels)
        yhat   = (scores >= thr_global).astype(int)
        prec = precision_score(labels, yhat, zero_division=0)
        rec  = recall_score(labels, yhat, zero_division=0)
        acc  = accuracy_score(labels, yhat)
        f1   = f1_score(labels, yhat, zero_division=0)
        ap   = average_precision_score(labels, scores) if labels.sum() > 0 else float("nan")
        pr_p, pr_r, _ = precision_recall_curve(labels, scores)
        auc_pr = auc(pr_r, pr_p) if len(pr_r) > 1 else float("nan")
        try: auc_roc = roc_auc_score(labels, scores) if (np.unique(labels).size == 2) else float("nan")
        except Exception: auc_roc = float("nan")

        print(f"\n[OVERALL TEST] thr={thr_global:.6f} | "
              f"Prec={prec:.3f} Rec={rec:.3f} Acc={acc:.3f} F1={f1:.3f} "
              f"AP={ap:.3f} AUC-PR={auc_pr:.3f} AUC-ROC={auc_roc:.3f}")


In [ ]:
@torch.no_grad()
def build_val_from_famset(
    cache,
    fams: list[str],
    encoder: torch.nn.Module,
    clf: torch.nn.Module,
    odin,
    rng: np.random.Generator,
    *,
    part: str = "val",
    k_per_family: int = 2000,
    steps: int = 1,
) -> tuple[dict[str, np.ndarray], dict[str, np.ndarray], np.ndarray, np.ndarray]:
    """
    Balanced K/K (OOD/ID) per family from `part`, score with anomaly score s = 1 - p_norm.
    Returns: (y_by_fam, s_by_fam, y_all, s_all)
    """
    y_by_fam, s_by_fam = {}, {}
    Ys, Ss = [], []
    for fam in sorted(fams):
        Xf, yf = _balanced_block(cache, fam, rng, part=part, cap=k_per_family)
        if Xf.size == 0:
            continue
        sf = _score_block_anomaly(Xf, encoder, clf, odin, steps=steps)  # s = 1 - p_norm
        y_by_fam[fam] = yf
        s_by_fam[fam] = sf
        Ys.append(yf); Ss.append(sf)

    if not Ys:
        return {}, {}, np.empty((0,), np.int64), np.empty((0,), np.float32)
    return y_by_fam, s_by_fam, np.concatenate(Ys), np.concatenate(Ss)


In [ ]:
from typing import Optional

def main(
    learn_odin: bool = True,
    *,
    baseline_T: float = 1.0,
    baseline_eps: float = 1.5e-3,
    choose_thr_by: str = "bestf1",   # "bestf1" | "fpr"
    target_fpr: float = 0.01,
    seed: int = 40,
    cap_per_family: int = 4000,
    val_k_per_family: int = 2000,
    resample_val_each_episode: bool = True,
    save_root: Optional[str] = None,
    data_dir: Optional[str] = None,
    n_episodes: int = N_EPISODES,    # control how many episodes
    outer_per_class_min: int = 100,  # 100–200 OOD per class
    outer_per_class_max: int = 200,
    lr_odin_cfg = 5e-4  # or 1e-4

):
    """
    Bilevel loop with:
      • held-out-only validation pool (K/K per family), scores = 1 - p_norm
      • global threshold picked on pooled held-out (bestF1 or fixed-FPR)
      • per-family table using per-family best-F1 thresholds
      • final test uses the best global threshold across episodes (by held-out F1)
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[DEVICE] Using device = {device}")

    # ------------- setup & data -------------
    set_seed(seed)
    if data_dir is None:
        # use existing global DATA_DIR if defined, else fallback to your old path
        data_dir = globals().get(
            "DATA_DIR",
            "/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_AL...d/",  # adjust if needed
        )

    files = discover_files_by_category(data_dir)
    if not files:
        raise RuntimeError(f"No '*.csv' found in {data_dir}")
    fams_all = sorted(files.keys())

    # HELDOUT_FAMILIES = [
    #   'Brute Force::Web',
    #   'Brute Force::XSS',
    #   'DDOS attack::HOIC',
    #   'Infilteration',
    #   'DoS attacks::GoldenEye',
    #   'FTP::BruteForce',
    #   'DoS attacks::Slowloris']

    HELDOUT_FAMILIES = [
    'Brute Force::Web',
    'DDOS attack::HOIC',
    'DoS attacks::GoldenEye',
    'FTP::BruteForce',
    'SQL Injection',
    ]


    heldout_resolved, _ = resolve_heldout(HELDOUT_FAMILIES, fams_all)
    heldout_fams = heldout_resolved
    seen_train_fams = [f for f in fams_all if f not in heldout_fams]
    print("Held-out (eval-only):", heldout_fams)
    print("Seen train families :", seen_train_fams)

    rng = np.random.default_rng(seed)
    cache = DataCache(files, max_rows=None)
    cache.load_all()
    cache.build_splits(rng, train_frac=2 / 3, val_frac=1 / 6, test_frac=1 / 6)
    cache.fit_scaler_on_train(rng, use_robust=True)
    cache.apply_scaler_all()

    # All non-held-out OOD families (cls=1 on train split)
    all_train_ood_fams = [
        f for f in cache.families_with(cls=1, part="train")
        if f in seen_train_fams
    ]
    print(f"[OUTER] Using {len(all_train_ood_fams)} OOD families (non-held-out) per episode")

    # ------------- models -------------
    D = cache.X[next(iter(cache.X))].shape[1]
    encoder = Encoder(
        in_dim=D, hid=ENC_HIDDEN, depth=ENC_DEPTH,
        p_drop=DROPOUT, bottleneck=EMBED_DIM, feat_drop=0.05
    ).to(device)
    decoder = Decoder(
        bottleneck=EMBED_DIM, hid=DEC_HIDDEN,
        out_dim=D, depth=DEC_DEPTH
    ).to(device)
    clf = Classifier(bottleneck=EMBED_DIM, num_classes=2).to(device)

    # ------------- ODIN: learn T only; eps fixed -------------
    if learn_odin:
        odin = ODINParams(
            T0=TEMP_INIT,
            eps0=EPS_INIT,
            learn_T=True,
            learn_eps=True,    # <<< only temperature is meta-learned
        ).to(device)
        lr_odin_cfg = 1e-3
        mode_blurb = "LEARNED-ODIN(T-only; eps fixed)"
        save_dir_root = "plots_perfam" if save_root is None else save_root
    else:
        odin = ODINParams(
            T0=baseline_T,
            eps0=baseline_eps,
            learn_T=False,
            learn_eps=False,
        ).to(device)
        for p in odin.parameters():
            p.requires_grad = False
        lr_odin_cfg = 0.0
        mode_blurb = f"FIXED-ODIN(T={baseline_T},eps={baseline_eps})"
        save_dir_root = "plots_perfam_fixed_odin" if save_root is None else save_root

    # ------------- trainers -------------
    steps_per_epoch_est_inner = 100
    total_inner_steps_est = int(n_episodes) * INNER_EPOCHS * steps_per_epoch_est_inner
    inner_trainer = InnerTrainer(
        encoder, decoder, clf,
        lr_max=INNER_LR_MAX,
        weight_decay=WEIGHT_DECAY,
        clip_norm=CLIP_NORM,
        use_amp=USE_AMP,
        lambda_rec=LAMBDA_REC,
        total_inner_steps_est=total_inner_steps_est,
    )

    steps_per_epoch_est_outer = 50
    total_outer_steps_est = int(n_episodes) * OUTER_EPOCHS * steps_per_epoch_est_outer
    # total_outer_steps_est unused but kept for symmetry / logging if you want

    # ID-anchored outer loss, T-only ODIN
    # outer_trainer = OuterTrainer(
    #     encoder=encoder,
    #     clf=clf,
    #     odin=odin,
    #     lr_odin=lr_odin_cfg,
    #     wd_odin=0.0,
    #     amp=USE_AMP,
    #     loss_mode="bce",   # ID=1, OOD=0 in outer loss
    #     pair_margin=0.5,   # kept for compatibility; unused in "bce" mode
    #     pair_max=2048,
    # )

    outer_trainer = OuterTrainer(
    encoder=encoder,
    clf=clf,
    odin=odin,
    lr_odin=lr_odin_cfg,
    wd_odin=0.0,
    amp=USE_AMP,
    loss_mode="bce",     # ID-anchored BCE
    pair_margin=0.5,
    pair_max=2048,
    train_clf=True,      # <<< enable φ updates in outer loss
)


    rng_epi = np.random.default_rng(seed + 777)

    # fixed per-episode per-family eval pools for overlays
    eval_pools = build_perfam_eval_pools(
        cache, fams_all, rng_epi, part="val",
        cap_per_family=cap_per_family,
    )

    # keep best held-out GLOBAL threshold picked across episodes
    best_val_f1 = -1.0
    best_val_thr = 0.5

    print(
        f"\n=== Running in mode: {mode_blurb} | "
        f"episodes={n_episodes} | outer_per_class=[{outer_per_class_min},{outer_per_class_max}] | "
        f"val_k_per_family={val_k_per_family} | choose_thr_by={choose_thr_by} ==="
    )

    # ================= EPISODES =================
    for ep in range(1, int(n_episodes) + 1):
        print(f"\n=========== EPISODE {ep:02d} ({mode_blurb}) ===========")

        # ---------- PRE ----------
        results_pre = eval_stage_per_family(
            eval_pools, encoder, clf, odin, steps=1, stage_name="pre"
        )

        # ---------- INNER (ID training) ----------
        META_TASKS = int(rng_epi.integers(1, 3))  # 1 or 2 inner tasks per episode
        meta_tasks = []
        inner_used_union: list[str] = []

        for _ in range(META_TASKS):
            inner_fams = select_inner_fams_up_to(
                cache,
                rng_epi,
                allowed_fams=seen_train_fams,
                part="train",
                inner_max_fams=INNER_MAX_FAMS,
                inner_min_fams=2,
            )
            if len(inner_fams) < 2:
                continue

            Xi, yi, used = build_inner_episode_batch(
                cache,
                rng_epi,
                part="train",
                fams=inner_fams,
                pos_per_fam=INNER_POS_PER_FAM,
                neg_per_fam=INNER_NEG_PER_FAM,
            )
            if Xi.size == 0:
                continue

            meta_tasks.append((Xi.astype(np.float32), yi.astype(np.int64), used))
            inner_used_union.extend(used)

        if len(meta_tasks) == 0:
            print("  (skip: no meta tasks built)")
            results_inner = {}
        else:
            print(
                f"[EP {ep:02d}] META inner: {len(meta_tasks)} tasks; "
                f"union_fams={sorted(set(inner_used_union))}"
            )
            inner_trainer.train_one_episode(
                meta_tasks,
                batch_size=INNER_BATCH,
                epochs=INNER_EPOCHS,
            )
            results_inner = eval_stage_per_family(
                eval_pools, encoder, clf, odin, steps=1, stage_name="inner"
            )

        # ---------- OUTER (all seen OOD fams, 100–200 per class, balanced) ----------
        Xq, yq, used_ood = build_outer_episode_batch(
            cache,
            rng_epi,
            ood_fams=all_train_ood_fams,
            part="train",
            per_class_min=outer_per_class_min,
            per_class_max=outer_per_class_max,
        )

        if Xq.size == 0:
            print("  (skip outer: empty outer batch)")
            results_outer = {}
        else:
            pos_ct = int(yq.sum())
            neg_ct = int(len(yq) - pos_ct)
            print(
                f"[EP {ep:02d}] OUTER | OOD={used_ood} | "
                f"X={Xq.shape} (pos={pos_ct}, neg={neg_ct})"
            )
            outer_trainer.train_one_episode(
                Xq=Xq.astype(np.float32),
                yq=yq.astype(np.int64),
                batch_size=OUTER_BATCH,
                epochs=OUTER_EPOCHS,
                steps=TRAIN_PGD_STEPS,
                shuffle=True,
                decoder=decoder,
            )
            results_outer = eval_stage_per_family(
                eval_pools, encoder, clf, odin, steps=1, stage_name="outer"
            )

        # ---------- PLOTS (per-family overlays) ----------
        try:
            plot_per_family_overlays(
                results_pre=results_pre,
                results_inner=results_inner,
                results_outer=results_outer,
                episode=ep,
                save_root=save_dir_root,
            )
        except Exception as e:
            print(f"[WARN] plot_per_family_overlays failed: {e}")

        # ---------- HELD-OUT VALIDATION ----------
        rng_val = (
            np.random.default_rng(seed + 10_000 + ep)
            if resample_val_each_episode
            else np.random.default_rng(seed + 10_000)
        )
        y_f_held, s_f_held, _, _ = build_val_from_famset(
            cache,
            heldout_fams,
            encoder,
            clf,
            odin,
            rng_val,
            part="val",
            k_per_family=val_k_per_family,
            steps=VAL_PGD_STEPS,
        )
        thr_star, f1_star = print_global_and_perfam(
            y_f_held,
            s_f_held,
            choose_thr_by=choose_thr_by,
            target_fpr=target_fpr,
            tag=f"VAL (HELD-OUT, EP {ep:02d})",
        )
        if np.isfinite(f1_star) and (f1_star > best_val_f1):
            best_val_f1 = f1_star
            best_val_thr = thr_star

        # ---------- TRAIN-FAMILY VALIDATION ----------
        rng_val_train = (
            np.random.default_rng(seed + 20_000 + ep)
            if resample_val_each_episode
            else np.random.default_rng(seed + 20_000)
        )
        y_f_train, s_f_train, _, _ = build_val_from_famset(
            cache,
            seen_train_fams,
            encoder,
            clf,
            odin,
            rng_val_train,
            part="val",
            k_per_family=val_k_per_family,
            steps=VAL_PGD_STEPS,
        )
        _thr_train, _ = print_global_and_perfam(
            y_f_train,
            s_f_train,
            choose_thr_by=choose_thr_by,
            target_fpr=target_fpr,
            tag=f"VAL (TRAIN-FAMS, EP {ep:02d})",
        )

    # ================= FINAL TEST =================
    print(
        f"\n[FINAL] Using best validation threshold across episodes: "
        f"thr*={best_val_thr:.6f} (F1={best_val_f1:.3f})"
    )

    rng_test = np.random.default_rng(seed + 888)

    final_test_balanced(
        cache,
        encoder,
        clf,
        odin,
        thr_global=float(best_val_thr),
        rng=rng_test,
    )



In [ ]:
#### Train T, esilon both

main(
    learn_odin=True,
    n_episodes=15,
    outer_per_class_min=5,
    outer_per_class_max=10,
)


[DEVICE] Using device = cuda
Held-out (eval-only): ['Brute Force::Web', 'DDOS attack::HOIC', 'DoS attacks::GoldenEye', 'FTP::BruteForce', 'SQL Injection']
Seen train families : ['Bot', 'Brute Force::XSS', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'Infilteration', 'SSH::Bruteforce']
[OUTER] Using 9 OOD families (non-held-out) per episode


/tmp/ipython-input-1241488025.py:115: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(



=== Running in mode: LEARNED-ODIN(T-only; eps fixed) | episodes=15 | outer_per_class=[5,10] | val_k_per_family=2000 | choose_thr_by=bestf1 ===

=========== EPISODE 01 (LEARNED-ODIN(T-only; eps fixed)) ===========
[EP 01] META inner: 1 tasks; union_fams=['Bot', 'Brute Force::XSS']
[INNER/BCE] Ep01 | Loss=1.13167 | lr=3.00e-03
[INNER/BCE] Ep02 | Loss=0.52579 | lr=3.00e-03
[INNER/BCE] Ep03 | Loss=0.40760 | lr=2.99e-03
[INNER/BCE] Ep04 | Loss=0.35880 | lr=2.99e-03
[INNER/BCE] Ep05 | Loss=0.48838 | lr=2.98e-03
[INNER/BCE] Ep06 | Loss=0.33465 | lr=2.97e-03
[INNER/BCE] Ep07 | Loss=0.26228 | lr=2.96e-03
[INNER/BCE] Ep08 | Loss=0.43299 | lr=2.95e-03
[INNER/BCE] Ep09 | Loss=0.20222 | lr=2.93e-03
[INNER/BCE] Ep10 | Loss=0.35552 | lr=2.92e-03
[EP 01] OUTER | OOD=['Brute Force::XSS', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Hulk', 'DoS attacks::Slowloris', 'Bot', 'Infilteration', 'DDOS attack::LOIC-UDP', 'SSH::Bruteforce', 'DDoS attacks::LOIC-HTTP'] | X=(134, 78) (pos=67, neg=67)


/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=4.66804 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep02 | Loss=4.58035 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=4.77768 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=4.78761 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=4.67485 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=4.71185 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=4.68695 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.

...

[OUTER/BCE] Ep02 | Loss=0.60702 T0=0.059904->T=0.059941  eps0=1.978346e-03->eps=1.978539e-03 ||∇rawT||=1.922e-01 ||∇raweps||=9.695e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=0.59866 T0=0.059904->T=0.059958  eps0=1.978346e-03->eps=1.978646e-03 ||∇rawT||=1.854e-01 ||∇raweps||=9.535e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=0.58754 T0=0.059904->T=0.059974  eps0=1.978346e-03->eps=1.978760e-03 ||∇rawT||=1.750e-01 ||∇raweps||=9.261e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=0.57466 T0=0.059904->T=0.059990  eps0=1.978346e-03->eps=1.978878e-03 ||∇rawT||=1.622e-01 ||∇raweps||=9.226e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=0.56022 T0=0.059904->T=0.060005  eps0=1.978346e-03->eps=1.979001e-03 ||∇rawT||=1.474e-01 ||∇raweps||=8.807e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.54486 T0=0.059904->T=0.060019  eps0=1.978346e-03->eps=1.979128e-03 ||∇rawT||=1.316e-01 ||∇raweps||=8.292e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep08 | Loss=0.52887 T0=0.059904->T=0.060032  eps0=1.978346e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=0.41731 T0=0.060103->T=0.060111  eps0=1.980091e-03->eps=1.980173e-03 ||∇rawT||=1.877e-01 ||∇raweps||=6.292e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep02 | Loss=0.42396 T0=0.060103->T=0.060119  eps0=1.980091e-03->eps=1.980203e-03 ||∇rawT||=1.813e-01 ||∇raweps||=2.353e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=0.45179 T0=0.060103->T=0.060127  eps0=1.980091e-03->eps=1.980186e-03 ||∇rawT||=2.180e-01 ||∇raweps||=2.372e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=0.40796 T0=0.060103->T=0.060136  eps0=1.980091e-03->eps=1.980127e-03 ||∇rawT||=1.756e-01 ||∇raweps||=2.355e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=0.40687 T0=0.060103->T=0.060144  eps0=1.980091e-03->eps=1.980030e-03 ||∇rawT||=1.792e-01 ||∇raweps||=2.331e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=0.37243 T0=0.060103->T=0.060153  eps0=1.980091e-03->eps=1.979900e-03 ||∇rawT||=1.491e-01 ||∇raweps||=2.277e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.37194 T0=0.060103->T=0.060161  eps0=1.980091e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=0.39479 T0=0.060215->T=0.060221  eps0=1.978105e-03->eps=1.978103e-03 ||∇rawT||=2.204e-01 ||∇raweps||=1.061e-03  lr_odin=1.00e-03
[OUTER/BCE] Ep02 | Loss=0.39784 T0=0.060215->T=0.060227  eps0=1.978105e-03->eps=1.978304e-03 ||∇rawT||=2.185e-01 ||∇raweps||=1.057e-03  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=0.39291 T0=0.060215->T=0.060234  eps0=1.978105e-03->eps=1.978691e-03 ||∇rawT||=2.137e-01 ||∇raweps||=1.070e-03  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=0.38469 T0=0.060215->T=0.060242  eps0=1.978105e-03->eps=1.979248e-03 ||∇rawT||=2.069e-01 ||∇raweps||=1.085e-03  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=0.37407 T0=0.060215->T=0.060250  eps0=1.978105e-03->eps=1.979960e-03 ||∇rawT||=1.991e-01 ||∇raweps||=1.101e-03  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=0.36311 T0=0.060215->T=0.060258  eps0=1.978105e-03->eps=1.980815e-03 ||∇rawT||=1.872e-01 ||∇raweps||=1.112e-03  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.32088 T0=0.060215->T=0.060266  eps0=1.978105e-03->eps=1.

In [ ]:
# #### Train T, esilon both

#### Train T, esilon both

main(
    learn_odin=True,
    n_episodes=15,
    outer_per_class_min=20,
    outer_per_class_max=20,
)



[DEVICE] Using device = cuda
Held-out (eval-only): ['Brute Force::Web', 'DDOS attack::HOIC', 'DoS attacks::GoldenEye', 'FTP::BruteForce', 'SQL Injection']
Seen train families : ['Bot', 'Brute Force::XSS', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'Infilteration', 'SSH::Bruteforce']
[OUTER] Using 9 OOD families (non-held-out) per episode


/tmp/ipython-input-1241488025.py:115: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(



=== Running in mode: LEARNED-ODIN(T-only; eps fixed) | episodes=15 | outer_per_class=[20,20] | val_k_per_family=2000 | choose_thr_by=bestf1 ===

=========== EPISODE 01 (LEARNED-ODIN(T-only; eps fixed)) ===========
[EP 01] META inner: 1 tasks; union_fams=['Bot', 'Brute Force::XSS']
[INNER/BCE] Ep01 | Loss=1.13167 | lr=3.00e-03
[INNER/BCE] Ep02 | Loss=0.52579 | lr=3.00e-03
[INNER/BCE] Ep03 | Loss=0.40760 | lr=2.99e-03
[INNER/BCE] Ep04 | Loss=0.35880 | lr=2.99e-03
[INNER/BCE] Ep05 | Loss=0.48838 | lr=2.98e-03
[INNER/BCE] Ep06 | Loss=0.33465 | lr=2.97e-03
[INNER/BCE] Ep07 | Loss=0.26228 | lr=2.96e-03
[INNER/BCE] Ep08 | Loss=0.43299 | lr=2.95e-03
[INNER/BCE] Ep09 | Loss=0.20222 | lr=2.93e-03
[INNER/BCE] Ep10 | Loss=0.35552 | lr=2.92e-03
[EP 01] OUTER | OOD=['Brute Force::XSS', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Hulk', 'DoS attacks::Slowloris', 'Bot', 'Infilteration', 'DDOS attack::LOIC-UDP', 'SSH::Bruteforce', 'DDoS attacks::LOIC-HTTP'] | X=(360, 78) (pos=180, neg=180)


/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=5.09787 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep02 | Loss=5.09536 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=5.31015 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=5.21218 T0=0.055556->T=0.055611  eps0=1.991981e-03->eps=1.989990e-03 ||∇rawT||=4.646e+00 ||∇raweps||=9.518e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=4.65086 T0=0.055556->T=0.055722  eps0=1.991981e-03->eps=1.986090e-03 ||∇rawT||=4.467e+00 ||∇raweps||=6.829e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=3.90024 T0=0.055556->T=0.055832  eps0=1.991981e-03->eps=1.982584e-03 ||∇rawT||=3.800e+00 ||∇raweps||=1.397e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=3.08900 T0=0.055556->T=0.055939  eps0=1.991981e-03->eps=1.

...

[OUTER/BCE] Ep02 | Loss=0.31988 T0=0.059958->T=0.059961  eps0=1.873757e-03->eps=1.869844e-03 ||∇rawT||=2.579e-02 ||∇raweps||=2.055e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=0.32305 T0=0.059958->T=0.059962  eps0=1.873757e-03->eps=1.868065e-03 ||∇rawT||=6.104e-03 ||∇raweps||=5.817e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=0.33179 T0=0.059958->T=0.059962  eps0=1.873757e-03->eps=1.866350e-03 ||∇rawT||=1.884e-02 ||∇raweps||=5.215e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=0.31097 T0=0.059958->T=0.059963  eps0=1.873757e-03->eps=1.864251e-03 ||∇rawT||=4.310e-02 ||∇raweps||=9.857e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=0.29155 T0=0.059958->T=0.059963  eps0=1.873757e-03->eps=1.861827e-03 ||∇rawT||=4.286e-03 ||∇raweps||=1.187e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.30712 T0=0.059958->T=0.059963  eps0=1.873757e-03->eps=1.859450e-03 ||∇rawT||=3.794e-02 ||∇raweps||=1.068e-03  lr_odin=1.00e-03
[OUTER/BCE] Ep08 | Loss=0.28380 T0=0.059958->T=0.059963  eps0=1.873757e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=0.51379 T0=0.059971->T=0.059978  eps0=1.833621e-03->eps=1.831031e-03 ||∇rawT||=2.158e-01 ||∇raweps||=3.541e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep02 | Loss=0.59423 T0=0.059971->T=0.059991  eps0=1.833621e-03->eps=1.829014e-03 ||∇rawT||=4.471e-01 ||∇raweps||=7.755e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=0.46601 T0=0.059971->T=0.060008  eps0=1.833621e-03->eps=1.827432e-03 ||∇rawT||=3.150e-01 ||∇raweps||=5.557e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=0.51723 T0=0.059971->T=0.060027  eps0=1.833621e-03->eps=1.826209e-03 ||∇rawT||=4.260e-01 ||∇raweps||=2.137e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=0.37965 T0=0.059971->T=0.060048  eps0=1.833621e-03->eps=1.825244e-03 ||∇rawT||=8.455e-02 ||∇raweps||=6.931e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=0.38213 T0=0.059971->T=0.060068  eps0=1.833621e-03->eps=1.824555e-03 ||∇rawT||=1.968e-01 ||∇raweps||=5.693e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.38085 T0=0.059971->T=0.060087  eps0=1.833621e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep02 | Loss=0.47460 T0=0.060198->T=0.060222  eps0=1.823439e-03->eps=1.824654e-03 ||∇rawT||=1.639e-01 ||∇raweps||=3.072e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=0.43725 T0=0.060198->T=0.060236  eps0=1.823439e-03->eps=1.825874e-03 ||∇rawT||=1.635e-01 ||∇raweps||=4.189e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=0.39790 T0=0.060198->T=0.060250  eps0=1.823439e-03->eps=1.827637e-03 ||∇rawT||=7.366e-02 ||∇raweps||=2.273e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=0.32876 T0=0.060198->T=0.060263  eps0=1.823439e-03->eps=1.829739e-03 ||∇rawT||=8.930e-03 ||∇raweps||=5.467e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=0.29236 T0=0.060198->T=0.060275  eps0=1.823439e-03->eps=1.832157e-03 ||∇rawT||=2.134e-02 ||∇raweps||=1.470e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.27647 T0=0.060198->T=0.060284  eps0=1.823439e-03->eps=1.834801e-03 ||∇rawT||=3.297e-02 ||∇raweps||=3.164e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep08 | Loss=0.27600 T0=0.060198->T=0.060292  eps0=1.823439e-03->eps=1.

In [ ]:
# #### Train T, esilon both

#### Train T, esilon both

main(
    learn_odin=True,
    n_episodes=15,
    outer_per_class_min=50,
    outer_per_class_max=50,
)



[DEVICE] Using device = cuda
Held-out (eval-only): ['Brute Force::Web', 'DDOS attack::HOIC', 'DoS attacks::GoldenEye', 'FTP::BruteForce', 'SQL Injection']
Seen train families : ['Bot', 'Brute Force::XSS', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'Infilteration', 'SSH::Bruteforce']
[OUTER] Using 9 OOD families (non-held-out) per episode

=== Running in mode: LEARNED-ODIN(T-only; eps fixed) | episodes=15 | outer_per_class=[50,50] | val_k_per_family=2000 | choose_thr_by=bestf1 ===

=========== EPISODE 01 (LEARNED-ODIN(T-only; eps fixed)) ===========


/tmp/ipython-input-1241488025.py:115: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(


[EP 01] META inner: 1 tasks; union_fams=['Bot', 'Brute Force::XSS']
[INNER/BCE] Ep01 | Loss=1.13167 | lr=3.00e-03
[INNER/BCE] Ep02 | Loss=0.52579 | lr=3.00e-03
[INNER/BCE] Ep03 | Loss=0.40760 | lr=2.99e-03
[INNER/BCE] Ep04 | Loss=0.35880 | lr=2.99e-03
[INNER/BCE] Ep05 | Loss=0.48838 | lr=2.98e-03
[INNER/BCE] Ep06 | Loss=0.33465 | lr=2.97e-03
[INNER/BCE] Ep07 | Loss=0.26228 | lr=2.96e-03
[INNER/BCE] Ep08 | Loss=0.43299 | lr=2.95e-03
[INNER/BCE] Ep09 | Loss=0.20222 | lr=2.93e-03
[INNER/BCE] Ep10 | Loss=0.35552 | lr=2.92e-03
[EP 01] OUTER | OOD=['Brute Force::XSS', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Hulk', 'DoS attacks::Slowloris', 'Bot', 'Infilteration', 'DDOS attack::LOIC-UDP', 'SSH::Bruteforce', 'DDoS attacks::LOIC-HTTP'] | X=(900, 78) (pos=450, neg=450)


/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=5.29410 T0=0.055556->T=0.055556  eps0=1.991981e-03->eps=1.991981e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=1.00e-03
[OUTER/BCE] Ep02 | Loss=4.22075 T0=0.055556->T=0.055936  eps0=1.991981e-03->eps=1.989688e-03 ||∇rawT||=3.047e+00 ||∇raweps||=2.464e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=2.42557 T0=0.055556->T=0.056278  eps0=1.991981e-03->eps=1.990050e-03 ||∇rawT||=1.431e+00 ||∇raweps||=4.187e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=1.89339 T0=0.055556->T=0.056572  eps0=1.991981e-03->eps=1.984420e-03 ||∇rawT||=1.372e+00 ||∇raweps||=2.901e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=1.51040 T0=0.055556->T=0.056832  eps0=1.991981e-03->eps=1.976739e-03 ||∇rawT||=7.881e-01 ||∇raweps||=8.924e-06  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=1.16377 T0=0.055556->T=0.057045  eps0=1.991981e-03->eps=1.970733e-03 ||∇rawT||=7.733e-01 ||∇raweps||=4.595e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.96587 T0=0.055556->T=0.057220  eps0=1.991981e-03->eps=1.

...

[OUTER/BCE] Ep01 | Loss=0.25595 T0=0.062051->T=0.062051  eps0=1.471897e-03->eps=1.475473e-03 ||∇rawT||=4.372e-02 ||∇raweps||=1.191e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep02 | Loss=0.23245 T0=0.062051->T=0.062058  eps0=1.471897e-03->eps=1.479524e-03 ||∇rawT||=9.981e-02 ||∇raweps||=1.425e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=0.22831 T0=0.062051->T=0.062097  eps0=1.471897e-03->eps=1.482619e-03 ||∇rawT||=8.368e-02 ||∇raweps||=1.060e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=0.21552 T0=0.062051->T=0.062145  eps0=1.471897e-03->eps=1.484482e-03 ||∇rawT||=4.648e-02 ||∇raweps||=2.643e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=0.20942 T0=0.062051->T=0.062177  eps0=1.471897e-03->eps=1.486552e-03 ||∇rawT||=1.352e-02 ||∇raweps||=3.038e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=0.20233 T0=0.062051->T=0.062191  eps0=1.471897e-03->eps=1.488613e-03 ||∇rawT||=2.234e-03 ||∇raweps||=8.997e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.19872 T0=0.062051->T=0.062195  eps0=1.471897e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=0.20369 T0=0.062111->T=0.062083  eps0=1.508723e-03->eps=1.510348e-03 ||∇rawT||=2.709e-02 ||∇raweps||=1.748e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep02 | Loss=0.15159 T0=0.062111->T=0.062049  eps0=1.508723e-03->eps=1.510345e-03 ||∇rawT||=1.504e-02 ||∇raweps||=1.782e-06  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=0.14026 T0=0.062111->T=0.062025  eps0=1.508723e-03->eps=1.508364e-03 ||∇rawT||=1.675e-02 ||∇raweps||=1.155e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=0.13500 T0=0.062111->T=0.062007  eps0=1.508723e-03->eps=1.505136e-03 ||∇rawT||=8.372e-03 ||∇raweps||=1.277e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=0.12590 T0=0.062111->T=0.061985  eps0=1.508723e-03->eps=1.502137e-03 ||∇rawT||=4.026e-02 ||∇raweps||=1.208e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=0.12389 T0=0.062111->T=0.061963  eps0=1.508723e-03->eps=1.500333e-03 ||∇rawT||=5.014e-02 ||∇raweps||=1.304e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.12277 T0=0.062111->T=0.061934  eps0=1.508723e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=0.16685 T0=0.061673->T=0.061668  eps0=1.492625e-03->eps=1.491884e-03 ||∇rawT||=1.167e-02 ||∇raweps||=5.292e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep02 | Loss=0.16593 T0=0.061673->T=0.061683  eps0=1.492625e-03->eps=1.490373e-03 ||∇rawT||=4.201e-02 ||∇raweps||=3.790e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep03 | Loss=0.16326 T0=0.061673->T=0.061698  eps0=1.492625e-03->eps=1.489241e-03 ||∇rawT||=1.833e-02 ||∇raweps||=2.560e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep04 | Loss=0.16040 T0=0.061673->T=0.061712  eps0=1.492625e-03->eps=1.487601e-03 ||∇rawT||=3.577e-02 ||∇raweps||=3.495e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep05 | Loss=0.15732 T0=0.061673->T=0.061738  eps0=1.492625e-03->eps=1.485416e-03 ||∇rawT||=4.425e-02 ||∇raweps||=1.528e-04  lr_odin=1.00e-03
[OUTER/BCE] Ep06 | Loss=0.15561 T0=0.061673->T=0.061756  eps0=1.492625e-03->eps=1.484232e-03 ||∇rawT||=3.661e-02 ||∇raweps||=1.256e-05  lr_odin=1.00e-03
[OUTER/BCE] Ep07 | Loss=0.15325 T0=0.061673->T=0.061766  eps0=1.492625e-03->eps=1.

In [ ]:
main(
    learn_odin=False,
    n_episodes=18,
    outer_per_class_min=5,
    outer_per_class_max=10,
)


[DEVICE] Using device = cuda
Held-out (eval-only): ['Brute Force::Web', 'DDOS attack::HOIC', 'DoS attacks::GoldenEye', 'FTP::BruteForce', 'SQL Injection']
Seen train families : ['Bot', 'Brute Force::XSS', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'Infilteration', 'SSH::Bruteforce']
[OUTER] Using 9 OOD families (non-held-out) per episode

=== Running in mode: FIXED-ODIN(T=1.0,eps=0.0015) | episodes=18 | outer_per_class=[5,10] | val_k_per_family=2000 | choose_thr_by=bestf1 ===

=========== EPISODE 01 (FIXED-ODIN(T=1.0,eps=0.0015)) ===========


/tmp/ipython-input-1241488025.py:115: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(


[EP 01] META inner: 1 tasks; union_fams=['Bot', 'Brute Force::XSS']
[INNER/BCE] Ep01 | Loss=1.14109 | lr=3.00e-03
[INNER/BCE] Ep02 | Loss=0.39671 | lr=3.00e-03
[INNER/BCE] Ep03 | Loss=0.32755 | lr=2.99e-03
[INNER/BCE] Ep04 | Loss=0.36332 | lr=2.99e-03
[INNER/BCE] Ep05 | Loss=0.53477 | lr=2.99e-03
[INNER/BCE] Ep06 | Loss=0.29710 | lr=2.98e-03
[INNER/BCE] Ep07 | Loss=1.20424 | lr=2.97e-03
[INNER/BCE] Ep08 | Loss=0.45447 | lr=2.96e-03
[INNER/BCE] Ep09 | Loss=0.32086 | lr=2.95e-03
[INNER/BCE] Ep10 | Loss=0.31239 | lr=2.94e-03
[EP 01] OUTER | OOD=['Brute Force::XSS', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Hulk', 'DoS attacks::Slowloris', 'Bot', 'Infilteration', 'DDOS attack::LOIC-UDP', 'SSH::Bruteforce', 'DDoS attacks::LOIC-HTTP'] | X=(134, 78) (pos=67, neg=67)


/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=3.45405 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep02 | Loss=3.40880 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=3.46084 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=3.52528 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=3.48852 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=3.45457 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=3.44703 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.

...

[OUTER/BCE] Ep02 | Loss=5.09462 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=5.31412 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=5.47841 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=5.26299 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=5.35245 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=5.31746 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep08 | Loss=5.10368 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep13 | Loss=4.28418 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep14 | Loss=4.30647 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep15 | Loss=4.11707 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[PLOT] Per-family overlays saved (14 figs) → plots_perfam_fixed_odin/ep17
[VAL-METRICS] VAL (HELD-OUT, EP 17) GLOBAL | thr*=0.000003 (best-F1) | Prec=0.653 Rec=0.869 Acc=0.703 F1=0.745 AP=0.787 PR-AUC=0.782 AUC-ROC=0.756
[VAL-METRICS] VAL (HELD-OUT, EP 17) PER-FAMILY @ best-F1 (class-specific thresholds)
  🧪 Brute Force::Web       | thr=0.000000 | Prec=0.500 Rec=1.000 Acc=0.500 F1=0.667 AP=0.552 AUC-ROC=0.403
  🧪 DDOS attack::HOIC      | thr=0.000000 | Prec=0.754 Rec=0.910 Acc=0.807 F1=0.825 AP=0.715 AUC-ROC=0.784
  🧪 DoS attacks::Gold

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=5.86654 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep02 | Loss=5.71503 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=5.64927 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=5.81495 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=5.77090 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=5.83436 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=5.78175 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.

In [ ]:
main(
    learn_odin=False,
    n_episodes=15,
    outer_per_class_min=20,
    outer_per_class_max=20,
)


[DEVICE] Using device = cuda
Held-out (eval-only): ['Brute Force::Web', 'DDOS attack::HOIC', 'DoS attacks::GoldenEye', 'FTP::BruteForce', 'SQL Injection']
Seen train families : ['Bot', 'Brute Force::XSS', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'Infilteration', 'SSH::Bruteforce']
[OUTER] Using 9 OOD families (non-held-out) per episode

=== Running in mode: FIXED-ODIN(T=1.0,eps=0.0015) | episodes=15 | outer_per_class=[20,20] | val_k_per_family=2000 | choose_thr_by=bestf1 ===

=========== EPISODE 01 (FIXED-ODIN(T=1.0,eps=0.0015)) ===========


/tmp/ipython-input-1241488025.py:115: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(


[EP 01] META inner: 1 tasks; union_fams=['Bot', 'Brute Force::XSS']
[INNER/BCE] Ep01 | Loss=1.13167 | lr=3.00e-03
[INNER/BCE] Ep02 | Loss=0.52579 | lr=3.00e-03
[INNER/BCE] Ep03 | Loss=0.40760 | lr=2.99e-03
[INNER/BCE] Ep04 | Loss=0.35880 | lr=2.99e-03
[INNER/BCE] Ep05 | Loss=0.48838 | lr=2.98e-03
[INNER/BCE] Ep06 | Loss=0.33465 | lr=2.97e-03
[INNER/BCE] Ep07 | Loss=0.26228 | lr=2.96e-03
[INNER/BCE] Ep08 | Loss=0.43299 | lr=2.95e-03
[INNER/BCE] Ep09 | Loss=0.20222 | lr=2.93e-03
[INNER/BCE] Ep10 | Loss=0.35552 | lr=2.92e-03
[EP 01] OUTER | OOD=['Brute Force::XSS', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Hulk', 'DoS attacks::Slowloris', 'Bot', 'Infilteration', 'DDOS attack::LOIC-UDP', 'SSH::Bruteforce', 'DDoS attacks::LOIC-HTTP'] | X=(360, 78) (pos=180, neg=180)
[OUTER/BCE] Ep01 | Loss=5.09788 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00


/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep02 | Loss=5.09537 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=5.31015 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=5.21217 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=5.22447 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=5.17692 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=5.00022 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep08 | Loss=5.03655 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.

...

[OUTER/BCE] Ep01 | Loss=9.03722 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep02 | Loss=9.20406 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=8.59883 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=9.11540 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=8.27344 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=8.78024 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=8.58215 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep02 | Loss=11.97726 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=12.87727 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=12.30760 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=12.33893 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=12.41387 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=11.91776 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep08 | Loss=12.51758 T0=0.055556->T=0.055556  eps0=1.491111e-03-

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=9.40006 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep02 | Loss=9.14497 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=9.68724 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=9.88522 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=9.69821 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=10.07806 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=9.73533 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1

In [ ]:
main(
    learn_odin=False,
    n_episodes=15,
    outer_per_class_min=50,
    outer_per_class_max=50,
)


[DEVICE] Using device = cuda
Held-out (eval-only): ['Brute Force::Web', 'DDOS attack::HOIC', 'DoS attacks::GoldenEye', 'FTP::BruteForce', 'SQL Injection']
Seen train families : ['Bot', 'Brute Force::XSS', 'DDOS attack::LOIC-UDP', 'DDoS attacks::LOIC-HTTP', 'DoS attacks::Hulk', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Slowloris', 'Infilteration', 'SSH::Bruteforce']
[OUTER] Using 9 OOD families (non-held-out) per episode

=== Running in mode: FIXED-ODIN(T=1.0,eps=0.0015) | episodes=15 | outer_per_class=[50,50] | val_k_per_family=2000 | choose_thr_by=bestf1 ===

=========== EPISODE 01 (FIXED-ODIN(T=1.0,eps=0.0015)) ===========


/tmp/ipython-input-1241488025.py:115: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(


[EP 01] META inner: 1 tasks; union_fams=['Bot', 'Brute Force::XSS']
[INNER/BCE] Ep01 | Loss=1.13167 | lr=3.00e-03
[INNER/BCE] Ep02 | Loss=0.52579 | lr=3.00e-03
[INNER/BCE] Ep03 | Loss=0.40760 | lr=2.99e-03
[INNER/BCE] Ep04 | Loss=0.35880 | lr=2.99e-03
[INNER/BCE] Ep05 | Loss=0.48838 | lr=2.98e-03
[INNER/BCE] Ep06 | Loss=0.33465 | lr=2.97e-03
[INNER/BCE] Ep07 | Loss=0.26228 | lr=2.96e-03
[INNER/BCE] Ep08 | Loss=0.43299 | lr=2.95e-03
[INNER/BCE] Ep09 | Loss=0.20222 | lr=2.93e-03
[INNER/BCE] Ep10 | Loss=0.35552 | lr=2.92e-03
[EP 01] OUTER | OOD=['Brute Force::XSS', 'DoS attacks::SlowHTTPTest', 'DoS attacks::Hulk', 'DoS attacks::Slowloris', 'Bot', 'Infilteration', 'DDOS attack::LOIC-UDP', 'SSH::Bruteforce', 'DDoS attacks::LOIC-HTTP'] | X=(900, 78) (pos=450, neg=450)


/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=5.29407 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep02 | Loss=5.28874 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=5.30335 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=5.31986 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=5.31169 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=5.29854 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=5.29788 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.

...

[OUTER/BCE] Ep01 | Loss=2.12251 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep02 | Loss=2.12338 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=2.10930 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=2.12203 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=2.12454 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=2.11323 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=2.11202 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=2.01146 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep02 | Loss=2.01197 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=2.00848 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=2.01036 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=2.00950 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=2.00699 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=2.01027 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.

/tmp/ipython-input-1241488025.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):
/tmp/ipython-input-1241488025.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.amp):


[OUTER/BCE] Ep01 | Loss=2.46802 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep02 | Loss=2.46951 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep03 | Loss=2.46879 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep04 | Loss=2.46744 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep05 | Loss=2.47105 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep06 | Loss=2.46999 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.491111e-03 ||∇rawT||=0.000e+00 ||∇raweps||=0.000e+00  lr_odin=0.00e+00
[OUTER/BCE] Ep07 | Loss=2.46910 T0=0.055556->T=0.055556  eps0=1.491111e-03->eps=1.

In [ ]:
# ### Train only T
# main(
#     learn_odin=True,
#     choose_thr_by="bestf1",
#     val_k_per_family=2000,
#     resample_val_each_episode=True
# )


In [ ]:
# ### Train only T
# main(
#     learn_odin=True,
#     choose_thr_by="fpr",
#     val_k_per_family=2000,
#     resample_val_each_episode=True
# )


In [ ]:
# main(
#     learn_odin=False,
#     baseline_T=1.0,
#     baseline_eps=1.5e-3,
#     choose_thr_by="fpr",
#     target_fpr=0.01,
#     val_k_per_family=2000,
#     resample_val_each_episode=True
# )

In [ ]:
# main(learn_odin=False, choose_thr_by="fpr", target_fpr=0.01, val_k_per_family=2000, resample_val_each_episode=False)


In [ ]:
# main(
#   learn_odin=False,
#   baseline_T=1.0,
#   baseline_eps=1.5e-3,
#   choose_thr_by="fpr"   # or "bestf1"
# )




In [ ]:
# main(
#   learn_odin=False,
#   baseline_T=1.0,
#   baseline_eps=1.5e-3,
#   choose_thr_by="bestf1"   # or "bestf1"
# )




In [ ]:
# import re
# import os
# import matplotlib.pyplot as plt
# import numpy as np

# # === Directory to save plots ===
# SAVE_DIR = "./val_metric_plots_run5555__NO_learn"
# os.makedirs(SAVE_DIR, exist_ok=True)

# # === Paste your log data here ===
# LOGS = r"""
# [VAL-GLOBAL] VAL (EP 01) | thr* (best-F1)=0.000001 | Prec=0.500 Rec=1.000 Acc=0.500 F1=0.667 AP=0.641 PR-AUC=0.639 AUC-ROC=0.475
# [VAL-GLOBAL] VAL (EP 02) | thr* (best-F1)=0.000113 | Prec=0.678 Rec=0.955 Acc=0.751 F1=0.793 AP=0.859 PR-AUC=0.858 AUC-ROC=0.857
# [VAL-GLOBAL] VAL (EP 03) | thr* (best-F1)=0.000187 | Prec=0.876 Rec=0.742 Acc=0.818 F1=0.803 AP=0.877 PR-AUC=0.876 AUC-ROC=0.810
# [VAL-GLOBAL] VAL (EP 04) | thr* (best-F1)=0.000100 | Prec=0.878 Rec=0.801 Acc=0.845 F1=0.838 AP=0.890 PR-AUC=0.890 AUC-ROC=0.842
# [VAL-GLOBAL] VAL (EP 05) | thr* (best-F1)=0.000087 | Prec=0.908 Rec=0.806 Acc=0.862 F1=0.854 AP=0.906 PR-AUC=0.908 AUC-ROC=0.855
# [VAL-GLOBAL] VAL (EP 06) | thr* (best-F1)=0.000015 | Prec=0.825 Rec=0.842 Acc=0.831 F1=0.833 AP=0.916 PR-AUC=0.919 AUC-ROC=0.891
# [VAL-GLOBAL] VAL (EP 07) | thr* (best-F1)=0.016784 | Prec=0.966 Rec=0.727 Acc=0.851 F1=0.830 AP=0.915 PR-AUC=0.917 AUC-ROC=0.884
# [VAL-GLOBAL] VAL (EP 08) | thr* (best-F1)=0.000031 | Prec=0.911 Rec=0.770 Acc=0.847 F1=0.835 AP=0.889 PR-AUC=0.896 AUC-ROC=0.832
# [VAL-GLOBAL] VAL (EP 09) | thr* (best-F1)=0.000145 | Prec=0.931 Rec=0.772 Acc=0.857 F1=0.844 AP=0.907 PR-AUC=0.912 AUC-ROC=0.870
# [VAL-GLOBAL] VAL (EP 11) | thr* (best-F1)=0.001440 | Prec=0.978 Rec=0.710 Acc=0.847 F1=0.822 AP=0.886 PR-AUC=0.895 AUC-ROC=0.836
# [VAL-GLOBAL] VAL (EP 12) | thr* (best-F1)=0.025540 | Prec=0.981 Rec=0.717 Acc=0.852 F1=0.828 AP=0.911 PR-AUC=0.914 AUC-ROC=0.876
# [VAL-GLOBAL] VAL (EP 13) | thr* (best-F1)=0.017769 | Prec=0.955 Rec=0.759 Acc=0.862 F1=0.846 AP=0.913 PR-AUC=0.921 AUC-ROC=0.881
# [VAL-GLOBAL] VAL (EP 14) | thr* (best-F1)=0.000060 | Prec=0.817 Rec=0.736 Acc=0.785 F1=0.774 AP=0.867 PR-AUC=0.873 AUC-ROC=0.824
# [VAL-GLOBAL] VAL (EP 15) | thr* (best-F1)=0.000044 | Prec=0.850 Rec=0.785 Acc=0.823 F1=0.816 AP=0.880 PR-AUC=0.881 AUC-ROC=0.825
# [VAL-GLOBAL] VAL (EP 16) | thr* (best-F1)=0.000000 | Prec=0.593 Rec=0.903 Acc=0.642 F1=0.716 AP=0.767 PR-AUC=0.769 AUC-ROC=0.721
# """

# # === Regex ===
# line_re = re.compile(
#     r"\bEP\s*(?P<ep>\d+)\b.*?thr\*=(?P<thr>[\d\.eE\-]+)"
#     r".*?Prec=(?P<prec>\d+\.\d+)\s+Rec=(?P<rec>\d+\.\d+)\s+Acc=(?P<acc>\d+\.\d+)\s+F1=(?P<f1>\d+\.\d+)"
#     r".*?AP=(?P<ap>\d+\.\d+)\s+PR-AUC=(?P<prauc>\d+\.\d+)\s+AUC-ROC=(?P<auc>\d+\.\d+)"
# )

# # === Parse ===
# rows = []
# for line in LOGS.splitlines():
#     m = line_re.search(line)
#     if not m:
#         continue
#     rows.append({
#         "ep": int(m.group("ep")),
#         "thr": float(m.group("thr")),
#         "prec": float(m.group("prec")),
#         "rec": float(m.group("rec")),
#         "acc": float(m.group("acc")),
#         "f1": float(m.group("f1")),
#         "ap": float(m.group("ap")),
#         "prauc": float(m.group("prauc")),
#         "auc": float(m.group("auc")),
#     })

# rows.sort(key=lambda d: d["ep"])
# episodes = [r["ep"] for r in rows]
# get_series = lambda key: np.array([r[key] for r in rows])

# metrics = [
#     ("prec",  "Precision"),
#     ("rec",   "Recall"),
#     ("acc",   "Accuracy"),
#     ("f1",    "F1 Score"),
#     ("ap",    "Average Precision"),
#     ("prauc", "PR-AUC"),
#     ("auc",   "AUC-ROC"),
#     ("thr",   "Threshold (thr*)"),
# ]

# # === Helper for nice Y-axis limits ===
# def set_smart_ylim(values):
#     vmin, vmax = np.min(values), np.max(values)
#     pad = 0.02 * (vmax - vmin)
#     plt.ylim(vmin - pad, vmax + pad)
#     step = round((vmax - vmin) / 6, 2)
#     if step > 0:
#         plt.yticks(np.arange(round(vmin, 2), round(vmax + step, 2), step))

# # === Generate and save ===
# for key, title in metrics:
#     vals = get_series(key)
#     plt.figure(figsize=(7, 4))
#     plt.plot(episodes, vals, marker="o", linewidth=2, color="#007acc")
#     plt.xlabel("Episode", fontsize=11)
#     plt.ylabel(title, fontsize=11)
#     plt.title(f"{title} vs Episode", fontsize=12, fontweight="bold")
#     plt.xticks(episodes)
#     set_smart_ylim(vals)
#     plt.grid(True, linestyle="--", alpha=0.6)
#     plt.tight_layout()

#     save_path = os.path.join(SAVE_DIR, f"{key}_vs_episode.png")
#     plt.savefig(save_path, dpi=300, bbox_inches="tight")
#     plt.close()
#     print(f"✅ Saved: {save_path}")

# print(f"\nAll plots saved in: {os.path.abspath(SAVE_DIR)}")


In [ ]:
# def main(
#     learn_odin: bool = True,          # False => fixed ODIN baseline
#     baseline_T: float = 1.0,
#     baseline_eps: float = 1.5e-3,
#     seed: int = 40,
#     cap_per_family: int = 4000,       # for overlay plots
#     val_cap_per_family: int = 25_000, # for global validation pooling (balanced per family)
#     overlay_bins: int = 60,
# ):
#     """
#     Minimal changes:
#       - Validation pool is RESAMPLED each episode (rng depends on episode).
#       - Best global threshold (by F1 on pooled normal-vs-anomaly val) is tracked
#         across episodes and USED at final test (no more 0.5 fallback).
#     Everything else (inner/outer training, per-family overlays, test_per_family) is unchanged.
#     """
#     import numpy as np
#     import torch
#     from sklearn.metrics import (
#         precision_score, recall_score, accuracy_score, f1_score,
#         average_precision_score, roc_auc_score, precision_recall_curve, auc
#     )

#     # ----------------- helpers (local to main) -----------------
#     def _balanced_block(cache, fam, rng, part: str, cap: int):
#         """Balanced (ID=0 vs OOD=1) block for one family."""
#         m = min(cache.count(fam, 1, part), cache.count(fam, 0, part), cap)
#         if m <= 0:
#             return np.empty((0, cache.X[fam].shape[1]), np.float32), np.empty((0,), np.int64)
#         Xa = cache.sample(fam, 1, part, m, rng)
#         Xn = cache.sample(fam, 0, part, m, rng)
#         Xf = np.vstack([Xa, Xn]).astype(np.float32, copy=False)
#         yf = np.concatenate([np.ones(m, np.int64), np.zeros(m, np.int64)])
#         return Xf, yf

#     @torch.no_grad()
#     def _pool_val_global(cache, encoder, clf, odin, rng, part: str, cap_per_family: int, steps: int = 1):
#         """Build pooled global (normal vs anomaly) validation set and scores."""
#         encoder.eval(); clf.eval(); odin.eval()
#         fams = sorted(cache.files.keys())
#         ys, ss = [], []

#         dev = next(clf.parameters()).device
#         for fam in fams:
#             Xf, yf = _balanced_block(cache, fam, rng, part=part, cap=cap_per_family)
#             if Xf.size == 0:
#                 continue
#             xb = torch.from_numpy(Xf).to(dev)
#             p_norm = odin_score_eval(xb, encoder, clf, odin, steps=steps)  # [B] in [0,1]
#             s = (1.0 - p_norm).cpu().numpy()  # anomaly score: higher = more anomalous
#             ys.append(yf); ss.append(s)

#         if not ys:
#             return np.empty(0, np.int64), np.empty(0, np.float32)
#         return np.concatenate(ys), np.concatenate(ss)

#     def _best_f1_threshold(y_true: np.ndarray, scores: np.ndarray) -> tuple[float, float]:
#         """Pick threshold maximizing F1 (using PR curve thresholds)."""
#         if y_true.size == 0:
#             return 0.5, float("nan")
#         P, R, thr = precision_recall_curve(y_true, scores)  # thr aligns with P[1:], R[1:]
#         # handle degenerate case
#         if thr.size == 0:
#             # fall back to median score
#             candidates = [float(np.median(scores))]
#             best_f1, best_thr = -1.0, candidates[0]
#         else:
#             F1 = (2 * P[1:] * R[1:]) / np.maximum(P[1:] + R[1:], 1e-12)
#             idx = int(np.nanargmax(F1))
#             best_f1 = float(F1[idx])
#             best_thr = float(thr[idx])
#         return best_thr, best_f1

#     def _metrics_at_threshold(y_true: np.ndarray, scores: np.ndarray, thr: float) -> dict:
#         yhat = (scores >= thr).astype(int)
#         prec = precision_score(y_true, yhat, zero_division=0)
#         rec  = recall_score(y_true, yhat, zero_division=0)
#         acc  = accuracy_score(y_true, yhat)
#         f1   = f1_score(y_true, yhat, zero_division=0)
#         try:
#             auc_roc = roc_auc_score(y_true, scores) if np.unique(y_true).size == 2 else float("nan")
#         except Exception:
#             auc_roc = float("nan")
#         ap = average_precision_score(y_true, scores) if y_true.sum() > 0 else float("nan")
#         try:
#             P, R, _ = precision_recall_curve(y_true, scores)
#             pr_auc = auc(R, P) if len(R) > 1 else float("nan")
#         except Exception:
#             pr_auc = float("nan")
#         return dict(prec=prec, rec=rec, acc=acc, f1=f1, auc_roc=auc_roc, ap=ap, pr_auc=pr_auc)

#     class _BestValTracker:
#         def __init__(self):
#             self.best_f1 = -1.0
#             self.best_thr = 0.5
#             self.best_ep = 0
#         def update(self, ep: int, thr: float, f1: float):
#             if np.isfinite(f1) and (f1 > self.best_f1):
#                 self.best_f1 = float(f1)
#                 self.best_thr = float(thr)
#                 self.best_ep = int(ep)

#     # ----------------- setup -----------------
#     set_seed(seed)

#     DATA_DIR = "/content/gdrive/MyDrive/DATASETS/CSE_CIC_IDS_ALL_DATA/CSE-CIC-IDS2018/Processed/"
#     files = discover_files_by_category(DATA_DIR)
#     if not files:
#         raise RuntimeError(f"No '*.csv' found in {DATA_DIR}")

#     fams_all = sorted(files.keys())
#     print("Discovered families (category::subcategory):")
#     for k in fams_all: print("  ", k)

#     HELDOUT_FAMILIES = ['Brute Force::Web','Brute Force::XSS','DDOS attack::HOIC','Infilteration']
#     heldout_resolved, _missing = resolve_heldout(HELDOUT_FAMILIES, fams_all)
#     heldout_fams    = heldout_resolved
#     seen_train_fams = [f for f in fams_all if f not in heldout_fams]
#     print("\nHeld-out (eval-only):", heldout_fams)
#     print("Seen train families :", seen_train_fams)

#     rng   = np.random.default_rng(42)
#     cache = DataCache(files, max_rows=None)
#     cache.load_all()
#     cache.build_splits(rng, train_frac=2/3, val_frac=1/6, test_frac=1/6)
#     cache.fit_scaler_on_train(rng, use_robust=True)
#     cache.apply_scaler_all()

#     # models
#     D = cache.X[next(iter(cache.X))].shape[1]
#     encoder = Encoder(in_dim=D, hid=ENC_HIDDEN, depth=ENC_DEPTH, p_drop=DROPOUT,
#                       bottleneck=EMBED_DIM, feat_drop=0.05).to(device)
#     decoder = Decoder(bottleneck=EMBED_DIM, hid=DEC_HIDDEN, out_dim=D, depth=DEC_DEPTH).to(device)
#     clf     = Classifier(bottleneck=EMBED_DIM, num_classes=2).to(device)

#     if learn_odin:
#         odin = ODINParams(
#             T0=TEMP_INIT, eps0=EPS_INIT,
#             learn_T=TUNE_TEMPERATURE, learn_eps=TUNE_TEMPERATURE,
#             T_bounds=(T_MIN, T_MAX), eps_bounds=(EPS_MIN, EPS_MAX)
#         ).to(device)
#         lr_odin_cfg   = 1e-3
#         save_dir_root = "plots_perfam"
#         mode_blurb    = "LEARNED-ODIN(T,eps)"
#     else:
#         odin = ODINParams(
#             T0=baseline_T, eps0=baseline_eps,
#             learn_T=False, learn_eps=False,
#             T_bounds=(T_MIN, T_MAX), eps_bounds=(EPS_MIN, EPS_MAX)
#         ).to(device)
#         for p in odin.parameters(): p.requires_grad = False
#         lr_odin_cfg   = 0.0
#         save_dir_root = "plots_perfam_fixed_odin"
#         mode_blurb    = f"FIXED-ODIN(T={baseline_T}, eps={baseline_eps})"

#     # trainers (unchanged)
#     steps_per_epoch_est_inner = 100
#     total_inner_steps_est = N_EPISODES * INNER_EPOCHS * steps_per_epoch_est_inner
#     inner_trainer = InnerTrainer(
#         encoder, decoder, clf,
#         lr_max=INNER_LR_MAX, weight_decay=WEIGHT_DECAY, clip_norm=CLIP_NORM,
#         use_amp=USE_AMP, lambda_rec=LAMBDA_REC,
#         total_inner_steps_est=total_inner_steps_est
#     )

#     steps_per_epoch_est_outer = 50
#     total_outer_steps_est = N_EPISODES * OUTER_EPOCHS * steps_per_epoch_est_outer
#     outer_trainer = OuterTrainer(
#         encoder, clf, odin,
#         lr_odin=lr_odin_cfg, lr_phi=3e-3,
#         wd_odin=0.0, wd_phi=1e-4,
#         total_outer_steps_est=total_outer_steps_est,
#         unfreeze_encoder=False,
#         enc_lr=1e-4, enc_wd=1e-4,
#         logT_l2=0.0, logeps_l2=0.0,
#         amp=USE_AMP,
#         margin_m=0.50, margin_alpha=0.10
#     )

#     # overlay pools & history (unchanged)
#     rng_epi = np.random.default_rng(seed + 777)
#     eval_pools = build_perfam_eval_pools(cache, fams_all, rng_epi, part="val", cap_per_family=cap_per_family)
#     perfam_history = PerFamMetricHistory(fam_names=fams_all)

#     # best validation threshold tracker
#     best = _BestValTracker()

#     print(f"\n=== Running in mode: {mode_blurb} | resample validation each episode; "
#           f'best global thr by F1 will be used at FINAL TEST ===')

#     # ============================= EPISODES =============================
#     for ep in range(1, N_EPISODES + 1):
#         print(f"\n=========== EPISODE {ep:02d} ({mode_blurb}) ===========")

#         # PRE (optional overlays)
#         results_pre = eval_stage_per_family(eval_pools, encoder, clf, odin, steps=1, stage_name="pre")

#         # INNER
#         META_TASKS = int(rng_epi.integers(1, 3))
#         meta_tasks = []
#         inner_used_union = []
#         for _ in range(META_TASKS):
#             inner_fams = select_inner_fams_up_to(
#                 cache, rng_epi, allowed_fams=seen_train_fams, part="train",
#                 inner_max_fams=INNER_MAX_FAMS, inner_min_fams=2
#             )
#             if len(inner_fams) < 2:
#                 continue
#             Xi, yi, used = build_inner_episode_batch(
#                 cache, rng_epi, part="train", fams=inner_fams,
#                 pos_per_fam=INNER_POS_PER_FAM, neg_per_fam=INNER_NEG_PER_FAM
#             )
#             if Xi.size == 0:
#                 continue
#             meta_tasks.append((Xi.astype(np.float32), yi.astype(np.int64), used))
#             inner_used_union.extend(used)

#         if not meta_tasks:
#             print("  (skip: no meta tasks built)");
#             continue

#         print(f"[EP {ep:02d}] META inner: {len(meta_tasks)} tasks; union_fams={sorted(set(inner_used_union))}")
#         inner_trainer.train_one_episode(meta_tasks, batch_size=INNER_BATCH, epochs=INNER_EPOCHS)

#         # INNER EVAL (optional overlays)
#         results_inner = eval_stage_per_family(eval_pools, encoder, clf, odin, steps=1, stage_name="inner")

#         # OUTER
#         allowed_for_outer = [f for f in seen_train_fams if (f not in heldout_fams) and (f not in inner_used_union)]
#         rng_epi.shuffle(allowed_for_outer)
#         ood_fams = allowed_for_outer[:min(TARGET_OOD_K, OUTER_MAX_FAMS, len(allowed_for_outer))]

#         if not ood_fams:
#             print("  (skip outer: no OOD families available)")
#             results_outer = {}
#         else:
#             Xq, yq, used_fams = build_outer_episode_batch(
#                 cache, rng_epi, part="train", ood_fams=ood_fams,
#                 fs_num_ood_classes=FS_NUM_OOD_CLASSES, fs_k_per_class=FS_K_PER_CLASS_OUTER,
#                 allow_smaller=FS_ALLOW_SMALLER,
#             )
#             if Xq.size == 0:
#                 print("  (skip outer: empty outer batch)")
#                 results_outer = {}
#             else:
#                 pos_ct = int(yq.sum()); neg_ct = int(len(yq) - pos_ct)
#                 print(f"[EP {ep:02d}] OUTER | OOD={used_fams} | X={Xq.shape} (pos={pos_ct}, neg={neg_ct})")
#                 outer_trainer.train_one_episode(
#                     Xq=Xq.astype(np.float32), yq=yq.astype(np.int64),
#                     batch_size=OUTER_BATCH, epochs=OUTER_EPOCHS,
#                     steps=TRAIN_PGD_STEPS, shuffle=True
#                 )
#                 results_outer = eval_stage_per_family(eval_pools, encoder, clf, odin, steps=1, stage_name="outer")

#         # PLOTS (unchanged)
#         plot_per_family_overlays(
#             results_pre=results_pre, results_inner=results_inner, results_outer=results_outer,
#             episode=ep, save_root=save_dir_root, bins=overlay_bins, verbose=False
#         )

#         # --------- GLOBAL VALIDATION (pooled normal vs anomaly), RESAMPLED EACH EP ---------
#         rng_val = np.random.default_rng(seed + 10_000 + ep)   # <-- resample pool per-episode
#         y_val, s_val = _pool_val_global(
#             cache, encoder, clf, odin, rng_val, part="val", cap_per_family=val_cap_per_family, steps=VAL_PGD_STEPS
#         )
#         if y_val.size == 0:
#             print(f"[VAL-METRICS] VAL (EP {ep:02d}) | no data")
#         else:
#             thr_star, f1_star = _best_f1_threshold(y_val, s_val)
#             stats = _metrics_at_threshold(y_val, s_val, thr_star)
#             print(f"[VAL-METRICS] VAL (EP {ep:02d}) | thr*={thr_star:.6f} | "
#                   f"Prec={stats['prec']:.3f} Rec={stats['rec']:.3f} Acc={stats['acc']:.3f} "
#                   f"F1={stats['f1']:.3f} AP={stats['ap']:.3f} PR-AUC={stats['pr_auc']:.3f} "
#                   f"AUC-ROC={stats['auc_roc']:.3f}")
#             best.update(ep, thr_star, stats['f1'])

#         # (Optional) per-family curves logging remains as in your code
#         # perfam_history.update(ep, perfam_stats) etc. if you want.

#         # cleanup
#         for (Xi, yi, _) in meta_tasks: del Xi, yi
#         if 'Xq' in locals(): del Xq, yq
#         gc.collect()

#     # ============================= FINAL TEST =============================
#     thr_final = float(best.best_thr)
#     print(f"\n[FINAL] Using best validation threshold across episodes: "
#           f"thr*={thr_final:.6f} (F1={best.best_f1:.3f} @ EP{best.best_ep})")

#     test_per_family(
#         cache, encoder, clf, odin,
#         thr_star=thr_final, rng=np.random.default_rng(seed + 888)
#     )


In [ ]:
# 1) Learn T, ε (ODIN learnable) — this is the default experiment
main(learn_odin=True, choose_thr_by="bestf1",
     seed=40, cap_per_family=4000, val_cap_per_family=25000)






# # 3) If you want a different random seed (everything else the same)
# main(learn_odin=True, seed=123)

# # 4) If you need faster VAL pooling during dev (smaller cap)
# main(learn_odin=True, val_cap_per_family=5000)


TypeError: main() got an unexpected keyword argument 'val_cap_per_family'

In [ ]:
# # # 2) Fixed ODIN baseline (no learning of T, ε)
# main(
#     learn_odin=False,
#     baseline_T=1.0,
#     baseline_eps=1.5e-3,
#     choose_thr_by="fpr",
#     target_fpr=0.01,
#     val_pos_per_class=2000
# )
